In [1]:
import mysql.connector
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox,simpledialog
import datetime
import re
from tkinter import scrolledtext
from twilio.rest import Client
import random 
import twilio


class SmartLibrary:   
    def __init__(self):
        self.T = tk.Tk()
        self.T.title('Home_page')
        self.Home = ttk.Frame(self.T ,borderwidth=6, relief="raised")
        self.Home.pack()
        self.Heading = tk.Label(self.Home, text='Smart Library', font=("Times New Roman", 18, 'bold'))
        self.Heading.grid(row=0, columnspan=3)
        self.admin = tk.Button(self.Home, text="Admin", font=("Times New Roman", 12), command=self.admin_login)
        self.admin.grid(row=1, column=3, padx=20, pady=20)
        self.student = tk.Button(self.Home, text="Student", font=("Times New Roman", 12), command=self.student_login)
        self.student.grid(row=4, column=3, padx=20, pady=20)
        self.T.mainloop()    
    def admin_login(self):
        self.T.destroy()
        self.T2=tk.Tk()
        self.T2.title("admin_login")
        front_page=ttk.Frame(self.T2,borderwidth=6, relief="raised")
        front_page.pack()
        user_name=tk.Label(front_page,text="User Name",font=("Times New Roman", 12))
        user_name.grid(row=0,column=0,padx=10,pady=10)
        user_name_entry=tk.Entry(front_page,width=30)
        user_name_entry.grid(row=0,column=1,padx=10,pady=10)
        password=tk.Label(front_page,text="Password",font=("Times New Roman", 12))
        password.grid(row=2,column=0,padx=10,pady=10)
        password_entry=tk.Entry(front_page,width=30,show="*")
        password_entry.grid(row=2,column=1,padx=10,pady=10)
        forgot_user_name=tk.Label(front_page,text='forgot_user_name',font=("Times New Roman", 12))
        forgot_user_name.grid(row=3,column=0,padx=10,pady=10)
        forgot_user_name.bind("<Enter>", lambda event: forgot_user_name.config(font=("Times New Roman", 12, "underline")))
        forgot_user_name.bind("<Leave>", lambda event: forgot_user_name.config(font=("Times New Roman", 11)))
        forgot_user_name.bind("<Button-1>", self.forget_user_name)
        back_button=tk.Button(front_page,text="Back to Home",font=("Times New Roman", 12),command=lambda:self.back_to_home(self.T2))
        back_button.grid(row=6,column=0,padx=10,pady=10,sticky="sw")
        forgot_password=tk.Label(front_page,text='forgot_password',font=("Times New Roman", 12))
        forgot_password.grid(row=3,column=1,padx=10,pady=10)
        forgot_password.bind("<Enter>", lambda event: forgot_password.config(font=("Times New Roman", 12, "underline")))
        forgot_password.bind("<Leave>", lambda event: forgot_password.config(font=("Times New Roman", 11)))
        forgot_password.bind("<Button-1>", self.forget_password)
        login_button=tk.Button(front_page,text="login",font=("Times New Roman", 12),command=lambda: self.check(user_name_entry, password_entry))
        login_button.grid(row=4,columnspan=4,padx=10,pady=10)
        self.T2.mainloop()
    def check(self,user_name_entry,password_entry):
        if not(user_name_entry.get()and password_entry.get()):
           messages="empty"
           simpledialog.messagebox.showinfo("error", messages)
           return
        user_name = user_name_entry.get()
        password = password_entry.get()
        try:
           mydb, cursor = self.establish_connection()
           query= "SELECT * FROM login_credentials WHERE login_id=%s AND password=%s"
           cursor.execute(query,(user_name,password))
           user =cursor.fetchone()
           if user:
            user_name_entry.delete(0, tk.END)
            password_entry.delete(0, tk.END)
            self.T2.withdraw()
            self.admin_page()
           else:
            simpledialog.messagebox.showinfo("Error", "no found")
            user_name_entry.delete(0, tk.END)
            password_entry.delete(0, tk.END)
            user_name_entry.focus_set()
        except mysql.connector.Error as err:
            print("Something went wrong: {}".format(err)) 
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()
    def admin_page(self):
        print("Admin_page")
        self.T3 = tk.Toplevel()
        self.T3.title("Admin_page")
        menubar=tk.Menu(self.T3)
        menubar_options=tk.Menu(menubar,tearoff=0)
        menubar_options.add_command(label="Issue books",command=lambda:self.issue_books(frame))
        menubar_options.add_command(label="Return books",command=lambda:self.return_books(frame))
        books_submenu=tk.Menu(menubar_options,tearoff=0)
        books_submenu.add_command(label="Add Book",command=lambda:self.add_books(frame))
        books_submenu.add_command(label="Update Book",command=lambda:self.update_books(frame))
        books_submenu.add_command(label="Delete/Remove Book",command=lambda:self.delete_book(frame))
        menubar_options.add_cascade(label="Books", menu=books_submenu)
        admin_submenu=tk.Menu(menubar,tearoff=0)
        admin_submenu.add_command(label="create_admin",command=lambda:self.create_new_admin(frame))
        admin_submenu.add_command(label="Update_admininfo",command=lambda:self.update_admin(frame))
        admin_submenu.add_command(label="Delete/Remove admin",command=lambda:self.delete_admin(frame))
        menubar_options.add_cascade(label="ADMIN",menu=admin_submenu)
        student_submenu=tk.Menu(menubar,tearoff=0)
        student_submenu.add_command(label="Create New Student",command=lambda:self.create_new_student(frame))
        student_submenu.add_command(label="Update StudentInfo",command=lambda:self.update_student(frame))
        student_submenu.add_command(label="Delete/Remove Student",command=lambda:self.delete_student(frame))
        student_submenu.add_command(label="Student History",command=lambda:self.get_student_history(frame))
        menubar_options.add_cascade(label="STUDENT",menu=student_submenu)
        menubar_options.add_command(label="Change password",command=self.change_password)
        menubar_options.add_command(label="Log_out",command=lambda:self.logout(self.T3))
        menubar.add_cascade(label="Menu", menu=menubar_options)
        self.T3.config(menu=menubar)
        frame=ttk.Frame(self.T3,borderwidth=6, relief="raised")
        frame.pack()
        label=tk.Label(frame,text="Login successful",font=("Times New Roman", 12))
        label.grid(rowspan=5,columnspan=5,padx=20,pady=20)
        self.T3.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.T3.mainloop()
    def issue_books(self,frame):
        self.clear_frame(frame)
        student_id=tk.Label(frame,text="Student_id",font=("Times New Roman", 12))
        student_id.grid(row=0,column=0,padx=10,pady=10)
        student_id_entry=tk.Entry(frame,width=30)
        student_id_entry.grid(row=0,column=1,padx=10,pady=10)
        book_id=tk.Label(frame,text="Book_Id",font=("Times New Roman", 12))
        book_id.grid(row=1,column=0,padx=10,pady=10)
        book_id_entry=tk.Entry(frame,width=30)
        book_id_entry.grid(row=1,column=1,padx=10,pady=10)
        Submit=tk.Button(frame,text="Issuse",font=("Times New Roman", 12),command=lambda:self.issue(student_id_entry, book_id_entry))
        Submit.grid(row=3,columnspan=2,padx=10,pady=10)
    def issue(self, student_id_entry, book_id_entry):
      errors = []
      if not (student_id_entry.get() and book_id_entry.get()):
          errors.append("All fields are required.")
      else:
       if len(student_id_entry.get()) != 10:
          errors.append("Student ID must be 10 characters long.")
       if not book_id_entry.get().isdigit():
          errors.append("Book ID must contain only integers.")
      
      if errors:
          tk.messagebox.showerror("Error", "\n".join(errors))
          student_id_entry.delete(0, tk.END)
          book_id_entry.delete(0, tk.END)
          student_id_entry.focus_set()
          return
      else:
       student_id_value=student_id_entry.get()
       book_id_value=book_id_entry.get()
       today = datetime.date.today()
       today_formatted = today.strftime("%d-%m-%Y")
       return_date = today + datetime.timedelta(days=15)
       return_date_formatted = return_date.strftime("%d-%m-%Y")
       try:
          mydb, cursor = self.establish_connection()
          cursor.execute("SELECT num_books_borrowed FROM student_database WHERE std_id = %s", (student_id_value,))
          num_books_borrowed = cursor.fetchone()[0]
          if num_books_borrowed >= 3:
             tk.messagebox.showinfo("Book Limit Exceeded", "The student has already borrowed the maximum number of books.")
             student_id_entry.delete(0, tk.END)
             book_id_entry.delete(0, tk.END)
             student_id_entry.focus_set()
             return
          cursor.execute("SELECT status FROM books_database WHERE book_id = %s", (book_id_value,))
          book_status = cursor.fetchone()
          if not book_status or book_status[0] != 'available':
             tk.messagebox.showinfo("Book Status", "The book is not available for issuing.")
             student_id_entry.delete(0, tk.END)
             book_id_entry.delete(0, tk.END)
             student_id_entry.focus_set()
             return
          cursor.execute("UPDATE books_database SET status = 'borrowed' WHERE book_id = %s", (book_id_value,))
          new_num_books_borrowed = num_books_borrowed + 1
          cursor.execute("UPDATE student_database SET num_books_borrowed = %s WHERE std_id = %s", (new_num_books_borrowed, student_id_value))
          query = "INSERT INTO issue_book ( student_id, book_id, issue_date, return_date) VALUES (%s, %s, %s, %s)"
          cursor.execute(query, ( student_id_value, book_id_value, today,return_date))
          mydb.commit()
          if cursor.rowcount > 0:
             info_add_book=f"Book issused  successfully \n Student_id ={student_id_value}\n Book _id={book_id_value}\n issue_Date={today_formatted}\n Return_Date ={return_date_formatted}"
             simpledialog.messagebox.showinfo("Book Issued", info_add_book)
             student_id_entry.delete(0, tk.END)
             book_id_entry.delete(0, tk.END)
             student_id_entry.focus_set()
          else:
             simpledialog.messagebox.showinfo("Update_info", "Books was not issued")
             student_id_entry.delete(0, tk.END)
             book_id_entry.delete(0, tk.END)
             student_id_entry.focus_set() 
       except mysql.connector.Error as error:  
          simpledialog.messagebox.showinfo("Book status Status", error)
          student_id_entry.delete(0, tk.END)
          book_id_entry.delete(0, tk.END)
          student_id_entry.focus_set() 
       finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()     
    def return_books(self,frame):
        self.clear_frame(frame)
        student_id=tk.Label(frame,text="Student_id",font=("Times New Roman", 12))
        student_id.grid(row=0,column=0,padx=10,pady=10)
        student_id_entry=tk.Entry(frame,width=30)
        student_id_entry.grid(row=0,column=1,padx=10,pady=10)
        book_id=tk.Label(frame,text="Book_Id",font=("Times New Roman", 12))
        book_id.grid(row=1,column=0,padx=10,pady=10)
        book_id_entry=tk.Entry(frame,width=30)
        book_id_entry.grid(row=1,column=1,padx=10,pady=10)
        Submit=tk.Button(frame,text="Return",font=("Times New Roman", 12),command=lambda:self.return_book_check(student_id_entry, book_id_entry))
        Submit.grid(row=3,columnspan=2,padx=10,pady=10)
    def return_book_check(self,student_id_entry, book_id_entry):
      errors = []
      if not (student_id_entry.get() and book_id_entry.get()):
          errors.append("All fields are required.")
      else:
       if len(student_id_entry.get()) != 10:
          errors.append("Student ID must be 10 characters long.")
       if not book_id_entry.get().isdigit():
          errors.append("Book ID must contain only integers.")
      if errors:
          tk.messagebox.showerror("Error", "\n".join(errors))
          student_id_entry.delete(0, tk.END)
          book_id_entry.delete(0, tk.END)
          student_id_entry.focus_set()
          return
      else:
       student_id_value=student_id_entry.get()
       book_id_value=book_id_entry.get()
       today = datetime.date.today()
       try:
          mydb, cursor = self.establish_connection()
          cursor.execute("SELECT student_id FROM issue_book WHERE book_id = %s", (book_id_entry.get(),))
          original_borrower_id = cursor.fetchone()[0]
          if original_borrower_id != student_id_entry.get():
            messagebox.showerror("Error", "Only the original borrower can return the book.")
            student_id_entry.delete(0, tk.END)
            book_id_entry.delete(0, tk.END)
            student_id_entry.focus_set()
            return
          cursor.execute("UPDATE books_database SET status = 'available' WHERE book_id = %s", (book_id_value,))
          cursor.execute("SELECT num_books_borrowed FROM student_database WHERE std_id = %s", (student_id_value,))
          num_books_borrowed = cursor.fetchone()[0]
          new_num_books_borrowed = max(num_books_borrowed - 1,0)
          cursor.execute("UPDATE student_database SET num_books_borrowed = %s WHERE std_id = %s", (new_num_books_borrowed, student_id_value))
          cursor.execute("SELECT return_date FROM issue_book WHERE student_id = %s",(student_id_value,))
          return_date=cursor.fetchone()[0]
          if today>return_date:
             print(return_date)
             print(today)
             difference = (today - return_date).days
             fine = max(difference * 2, 0) 
             cursor.execute("UPDATE issue_book SET fine = %s,return_on= %s WHERE student_id = %s", (fine,today, student_id_value))
             fine_message = f"Fine: ${fine}"
             print(fine)
          else:
             fine_message = "No fine"
             cursor.execute("UPDATE issue_book SET return_on= %s WHERE student_id = %s", (today, student_id_value))
          mydb.commit()
          if cursor.rowcount > 0:
             simpledialog.messagebox.showinfo("Return Status",fine_message)
             student_id_entry.delete(0, tk.END)
             book_id_entry.delete(0, tk.END)
             student_id_entry.focus_set()
          else:
             simpledialog.messagebox.showinfo("Return_status", "Already returned")
             student_id_entry.delete(0, tk.END)
             book_id_entry.delete(0, tk.END)
             student_id_entry.focus_set() 
       except mysql.connector.Error as error: 
          simpledialog.messagebox.showinfo("Book status Status", error)
          student_id_entry.delete(0, tk.END)
          book_id_entry.delete(0, tk.END)
          student_id_entry.focus_set() 
       finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()    
    def add_books(self,frame):
        self.clear_frame(frame)
        book_id_label=tk.Label(frame,text="Book Id",font=("Times New Roman", 12))
        book_id_label.grid(row=0,column=0,padx=10, pady=10)
        book_id=tk.Entry(frame,width=30)
        book_id.grid(row=0,column=1,padx=10,pady=10)
        book_name_label=tk.Label(frame,text="Book Name",font=("Times New Roman", 12))
        book_name_label.grid(row=1,column=0,padx=10,pady=10)
        book_name=tk.Entry(frame,width=30)
        book_name.grid(row=1,column=1,padx=10,pady=10)
        book_author_label=tk.Label(frame,text="Author_Name",font=("Times New Roman", 12))
        book_author_label.grid(row=2,column=0,padx=10,pady=10)
        book_author=tk.Entry(frame,width=30)
        book_author.grid(row=2,column=1,padx=10,pady=10)
        book_published_year_label=tk.Label(frame,text="Published Year",font=("Times New Roman", 12))
        book_published_year_label.grid(row=3,column=0,padx=10,pady=10)
        book_published_year=tk.Entry(frame,width=20)
        book_published_year.grid(row=3,column=1,padx=10,pady=10)
        add_book=tk.Button(frame,text="Add Book",font=("Times New Roman", 12),command=lambda:self.add_book_button(book_id,book_name,book_author,book_published_year))
        add_book.grid(row=5,columnspan=2,padx=10,pady=10)  
    def add_book_button(self,book_id,book_name,book_author,book_published_year):
        errors_add_book=[]
        if not (book_id.get() and book_name.get() and book_author.get() and book_published_year.get()):
          errors_add_book.append("All fields are required.")
        else:
          if not book_id.get().isdigit():
           errors_add_book.append("Book id  must contain only integers.")
          if not re.match("^[a-zA-Z\s\d\W]+$", book_name.get()):
            errors_add_book.append("Book title name must contain only alphabets.")
          if not re.match("^[a-zA-Z\s\d\W]+$", book_author.get()):
            errors_add_book.append("Book Author name must contain only alphabets.")
          if not book_published_year.get().isdigit():
           errors_add_book.append("Year must contain only integers.")
        if errors_add_book:
           tk.messagebox.showerror("Error", "\n".join(errors_add_book))
           book_id.delete(0, tk.END)
           book_name.delete(0, tk.END)
           book_author.delete(0, tk.END)
           book_published_year.delete(0, tk.END)
           book_id.focus_set()
           return
        else:
         book_id_val=book_id.get()
         book_name_val=book_name.get()
         book_author_val=book_author.get()
         book_published_year_val=book_published_year.get()
         try:
            mydb, cursor = self.establish_connection()
            cursor = mydb.cursor()
            query = "INSERT INTO books_database (book_id, book_name, book_author, book_published_year) VALUES (%s, %s, %s, %s)"
            cursor.execute(query, (book_id_val, book_name_val, book_author_val, book_published_year_val))
            mydb.commit()
            if cursor.rowcount > 0:
             info_add_book=f"Book Added  successfully \n Book Id ={book_id_val}\n Book Name={book_name_val}\n Book Author={book_author_val}\n Book Published Year ={book_published_year_val}"
             simpledialog.messagebox.showinfo("Book Issued", info_add_book)
             book_id.delete(0, tk.END)
             book_name.delete(0, tk.END)
             book_author.delete(0, tk.END)
             book_published_year.delete(0, tk.END)
             book_id.focus_set()
            else:
             simpledialog.messagebox.showinfo("Update_info", "Books was not added")
             book_id.delete(0, tk.END)
             book_name.delete(0, tk.END)
             book_author.delete(0, tk.END)
             book_published_year.delete(0, tk.END)
             book_id.focus_set()
         except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("Book status Status", error)
            book_id.delete(0, tk.END)
            book_name.delete(0, tk.END)
            book_author.delete(0, tk.END)
            book_published_year.delete(0, tk.END)
            book_id.focus_set()  
         finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()     
    def update_books(self,frame):
      self.clear_frame(frame)
      var_book_name = tk.BooleanVar()
      book_name = tk.Checkbutton(frame, text="book_name", variable=var_book_name)
      book_name.grid(row=2, column=1)
      var_book_author = tk.BooleanVar()
      book_author = tk.Checkbutton(frame, text="book_author", variable=var_book_author)
      book_author.grid(row=4, column=1)
      var_book_published_year = tk.BooleanVar()
      book_published_year = tk.Checkbutton(frame, text="book_published_year", variable=var_book_published_year)
      book_published_year.grid(row=6, column=1)
      button = tk.Button(frame, text="submit", command=lambda:self.book_call(frame,var_book_name,var_book_author,var_book_published_year))
      button.grid(row=8, column=1)
    def book_call(self,frame,var_book_name,var_book_author,var_book_published_year):
       self.disable_widgets(frame)
       self.checkbox_value(var_book_name,var_book_author,var_book_published_year,frame)
    def checkbox_value(self,var_book_name,var_book_author,var_book_published_year,frame):
       values = []
       #book_id, book_name, book_author, book_published_year, status
       if var_book_name.get():
           values.append("book_name")
       if var_book_author.get():
           values.append("book_author")
       if var_book_published_year.get():
           values.append("book_published_year")
       print("Selected values are:", values)
       self.creating_widgets(values,frame)
    def creating_widgets(self,values,frame):
       self.clear_frame(frame)
       row=1
       book_id=tk.Label(frame, text="Book_id",font=("Times New Roman", 12))
       book_id.grid(row=0, column=0,padx=10,pady=10)
       book_id_entry = tk.Entry(frame, width=30)
       book_id_entry.grid(row=0, column=1,padx=10,pady=10)
       entry_widgets=[]
       for name in values:
        name_label=tk.Label(frame, text=name,font=("Times New Roman", 12))
        name_label.grid(row=row, column=0)
        entry=tk.Entry(frame, width=30)
        entry.grid(row=row, column=1,padx=10,pady=10)
        entry_widgets.append(entry)
        row += 1
       submit = tk.Button(frame, text=" Update ", command=lambda:self.update_query(values,entry_widgets, book_id_entry))
       submit.grid(row=row, columnspan=2,padx=10,pady=10)
    def update_query(self,values, entry_widgets,book_id_entry):
       received_book_id=book_id_entry.get()
       #book_id, book_name, book_author, book_published_year, status
       errors_add_book=[]
       for entry in entry_widgets:
         if not entry.get():
            errors_add_book.append("Please fill in all the entry fields.")
       if not received_book_id:
          errors_add_book.append("Book_id Can't be Empty")  
       for name,entry in zip(values, entry_widgets):
        if name=="book_name":
            if not re.match("^[a-zA-Z\s\W]+$", entry.get()):
                errors_add_book.append("Book title name must contain only alphabets.")
        if name=="book_author":
              if not re.match("^[a-zA-Z\s\W]+$", entry.get()):
                errors_add_book.append("Book Author name must contain only alphabets.")
        if name=="book_published_year":
             if not entry.get().isdigit():
                  errors_add_book.append("Year must contain only integers.")
       if errors_add_book:
          tk.messagebox.showerror("Error", "\n".join(errors_add_book)) 
          for entry in entry_widgets:
            entry.delete(0,tk.END)
          book_id_entry.delete(0,tk.END)
       else: 
        set_clauses = []
        for value, entry in zip(values, entry_widgets):
          entry_value = entry.get()
          set_clause = f"{value}='{entry_value}'"  
          set_clauses.append(set_clause)
        set_values = ", ".join(set_clauses)
        try:
            mydb, cursor = self.establish_connection()
            cursor = mydb.cursor()
            query = f"UPDATE books_database SET {set_values} WHERE book_id = '{received_book_id}'"
            cursor.execute(query)
            mydb.commit()
            if cursor.rowcount > 0:
              info=f"Updated Book details:{set_values}"
              simpledialog.messagebox.showinfo("STUDENT Status", info)
              for entry in entry_widgets:
                 entry.delete(0,tk.END)
              book_id_entry.delete(0,tk.END)
            else:
              info="Unable to update the books data"
              simpledialog.messagebox.showinfo("STUDENT Status", info)
              for entry in entry_widgets:
                 entry.delete(0,tk.END)
              book_id_entry.delete(0,tk.END)     
        except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("STUDENT Status", error)
            for entry in entry_widgets:
                 entry.delete(0,tk.END)
            book_id_entry.delete(0,tk.END)
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()  
    def delete_book(self,frame):
       self.clear_frame(frame)
       book_id_label=tk.Label(frame,text="Book ID",font=("Times New Roman", 12))
       book_id_label.grid(row=0,column=0,padx=10,pady=10)
       book_id_entry=tk.Entry(frame,width=30)
       book_id_entry.grid(row=0,column=1,padx=10,pady=10)
       delete_button=tk.Button(frame,text="DELETE",font=("Times New Roman", 12),command=lambda:self.delete_book_check(book_id_entry))
       delete_button.grid(row=2,columnspan=3,padx=10,pady=10)
    def delete_book_check(self,book_id_entry):
        received_book_id = book_id_entry.get()
        if not received_book_id:
           messages="Book id cant be empty"
           simpledialog.messagebox.showinfo("error", messages)
           return
        try:
           mydb, cursor = self.establish_connection()
           query = "DELETE FROM books_database WHERE book_id = %s"
           cursor.execute(query, (received_book_id,))
           mydb.commit()
           if cursor.rowcount > 0:
              info="Record/Book Has been deleted/removed Successfully"
              simpledialog.messagebox.showinfo("Delete Status",info)
              book_id_entry.delete(0, tk.END)
              book_id_entry.focus_set()
           else:
              info="Unable to Delete the Record"
              simpledialog.messagebox.showinfo("Delete Status",info)
              book_id_entry.delete(0, tk.END)
              book_id_entry.focus_set()
        except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("Delete Book Status", error)
            book_id_entry.delete(0, tk.END)
            book_id_entry.focus_set()
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()  
    def create_new_admin(self,frame):
        self.clear_frame(frame)
        name_label=tk.Label(frame,text="Name",font=("Times New Roman", 12))
        name_label.grid(row=0,column=0,padx=10,pady=10)
        name=tk.Entry(frame,width=30)
        name.grid(row=0,column=1,padx=10,pady=10)
        user_name_label=tk.Label(frame,text="User Name",font=("Times New Roman", 12))
        user_name_label.grid(row=1,column=0,padx=10,pady=10)
        user_name=tk.Entry(frame,width=30)
        user_name.grid(row=1,column=1,padx=10,pady=10)
        phone_number_label=tk.Label(frame,text="Phone Number",font=("Times New Roman", 12))
        phone_number_label.grid(row=2,column=0,padx=10,pady=10)
        phone_number=tk.Entry(frame,width=30)
        phone_number.grid(row=2,column=1,padx=10,pady=10)
        gender_label=tk.Label(frame,text="gender",font=("Times New Roman", 12))
        gender_label.grid(row=3,column=0,padx=10,pady=10)
        gender=tk.Entry(frame,width=30)
        gender.grid(row=3,column=1,padx=10,pady=10)
        address_label=tk.Label(frame,text="Addres",font=("Times New Roman", 12))
        address_label.grid(row=4,column=0,padx=10,pady=10)
        address=tk.Entry(frame,width=30)
        address.grid(row=4,column=1,padx=10,pady=10)
        creat_admin=tk.Button(frame,text="Create admin",font=("Times New Roman", 12),command=lambda:self.create_admin_button(name,user_name,phone_number,gender,address))
        creat_admin.grid(row=6,columnspan=2,padx=10,pady=10)
    def create_admin_button(self,name,user_name,phone_number,gender,address):
        error_create_admin=[]
        if not (name.get() and user_name.get() and phone_number.get() and gender.get(),address.get()):
          error_create_admin.append("All fields are required.")
        else:
           if not name.get().isalpha():
             error_create_admin.append("Name name must contain only alphabets.")
           if not phone_number.get().isdigit():
             error_create_admin.append("Phone Number must contain only integers.")
           if len(phone_number.get()) != 10:
             error_create_admin.append("Phone Number  must be 10 characters long.")
           if not gender.get().isalpha():
              error_create_admin.append("gender must be only male or female")
        if error_create_admin:
           tk.messagebox.showerror("Error", "\n".join(error_create_admin))
           name.delete(0, tk.END)
           user_name.delete(0, tk.END)
           phone_number.delete(0, tk.END)
           gender.delete(0,tk.END)
           address.delete(0, tk.END)
           name.focus_set()
           return
        else:
           name_value=name.get()
           user_name_value=user_name.get()
           phone_number_value=phone_number.get()
           gender_value=gender.get()
           address_value=address.get()
        try:
            mydb, cursor = self.establish_connection()
            cursor = mydb.cursor()
            query = "INSERT INTO admin_table (name, user_name, phone_number, gender,address) VALUES (%s, %s, %s, %s,%s)"
            cursor.execute(query, (name_value, user_name_value, phone_number_value, gender_value,address_value))
            mydb.commit()
            if cursor.rowcount > 0:
             info=f"Admin Added successfully \n Name ={name_value}\n User_Name={user_name_value}\n Phone Number={phone_number_value}\n gender ={gender_value} \n Address={address_value}"
             simpledialog.messagebox.showinfo("Admin Status", info)
             name.delete(0, tk.END)
             user_name.delete(0, tk.END)
             phone_number.delete(0, tk.END)
             gender.delete(0, tk.END)
             address.delete(0, tk.END)
             name.focus_set()
            else:
               info="Unable to add the Admin"
               name.delete(0, tk.END)
               user_name.delete(0, tk.END)
               phone_number.delete(0, tk.END)
               gender.delete(0, tk.END)
               address.delete(0, tk.END)
               name.focus_set()
        except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("Admin Status", error)
            name.delete(0, tk.END)
            user_name.delete(0, tk.END)
            phone_number.delete(0, tk.END)
            gender.delete(0, tk.END)
            address.delete(0, tk.END)
            name.focus_set()
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()  
    def update_admin(self,frame):
      self.clear_frame(frame)
      var_name = tk.BooleanVar()
      name = tk.Checkbutton(frame, text="name", variable=var_name)
      name.grid(row=2, column=1)
      var_phone_number = tk.BooleanVar()
      phone_number = tk.Checkbutton(frame, text="phone_number", variable=var_phone_number)
      phone_number.grid(row=4, column=1)
      var_address=tk.BooleanVar()
      address=tk.Checkbutton(frame, text="address", variable=var_address)
      address.grid(row=6, column=1)
      var_gender = tk.BooleanVar()
      gender = tk.Checkbutton(frame, text="gender", variable=var_gender)
      gender.grid(row=8, column=1)
      button = tk.Button(frame, text="submit", command=lambda:self.admin_call(frame,var_name,var_phone_number,var_address,var_gender))
      button.grid(row=10, column=1)
    def admin_call(self,frame,var_name,var_phone_number,var_address,var_gender):
       self.disable_widgets(frame)
       self.checkbox_admin_value(var_name,var_phone_number,var_address,var_gender,frame)
    def checkbox_admin_value(self,var_name,var_phone_number,var_address,var_gender,frame):
       #name, user_name, phone_number, address, gender
       values_admin = []
       if var_name.get():
           values_admin.append("name")
       if var_phone_number.get():
           values_admin.append("phone_number")
       if var_address.get():
          values_admin.append("address")
       if var_gender.get():
           values_admin.append("gender")
       self.creating_admin_widgets(values_admin,frame)
    def creating_admin_widgets(self,values_admin,frame):
       self.clear_frame(frame)
       row=1
       user_name=tk.Label(frame, text="user_nmae",font=("Times New Roman", 12))
       user_name.grid(row=0, column=0,padx=10,pady=10)
       user_name_entry = tk.Entry(frame, width=30)
       user_name_entry.grid(row=0, column=1,padx=10,pady=10)
       entry_admin_widgets=[]
       for name in values_admin:
        name_label=tk.Label(frame, text=name,font=("Times New Roman", 12))
        name_label.grid(row=row, column=0)
        entry=tk.Entry(frame, width=30)
        entry.grid(row=row, column=1,padx=10,pady=10)
        entry_admin_widgets.append(entry)
        row += 1
       submit = tk.Button(frame, text=" Update ", command=lambda:self.update_admin_query(values_admin,entry_admin_widgets, user_name_entry))
       submit.grid(row=row, columnspan=2,padx=10,pady=10)
    def update_admin_query(self,values_admin, entry_admin_widgets,user_name_entry):
       received_user_name=user_name_entry.get()
              #name, user_name, phone_number, address, gender
       admin_errors=[]
       if not received_user_name:
          admin_errors.append("user_name Can't be Empty")
       for entry in entry_admin_widgets:
         if not entry.get():
            admin_errors.append("Please fill in all the entry fields.")
       for name,entry in zip(values_admin, entry_admin_widgets):
          if name=='name':
             if not re.match("^[a-zA-Z\s\W]+$", entry.get()):
                admin_errors.append("Name  must contain only alphabets")
          if name=="phone_number":
             if not entry.get().isdigit():
                admin_errors.append("Phone Number must contain only integers.")
             if len(entry.get()) != 10:
                admin_errors.append("Phone Number  must be 10 characters long.")    
          if name=="gender":
             if not entry.get().isalpha():
                admin_errors.append("gender must be only male or female") 
       if admin_errors:
          tk.messagebox.showerror("Error", "\n".join(admin_errors))
          user_name_entry.delete(0,tk.END)
          for entry in entry_admin_widgets:
             entry.delete(0,tk.END)
       else: 
        set_admin_clauses = []
        for value, entry in zip(values_admin, entry_admin_widgets):
          entry_value = entry.get()
          set_clause = f"{value}='{entry_value}'"  
          set_admin_clauses.append(set_clause)
        set_values = ", ".join(set_admin_clauses)
        try:
            mydb, cursor = self.establish_connection()
            cursor = mydb.cursor()
            query = f"UPDATE admin_table SET {set_values} WHERE user_name = '{received_user_name}'"
            cursor.execute(query)
            mydb.commit()
            if cursor.rowcount > 0:
             info_admin=f"Updated Admin info \n {set_values} "
             simpledialog.messagebox.showinfo("Admin Update status ", info_admin)
             user_name_entry.delete(0,tk.END)
             for entry in entry_admin_widgets:
                entry.delete(0,tk.END)
            else:
             info_admin="unable to Update the admin info"
             simpledialog.messagebox.showinfo("Admin Update status ", info_admin)
             user_name_entry.delete(0,tk.END)
             for entry in entry_admin_widgets:
                entry.delete(0,tk.END)
        except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("STUDENT Status", error)
            user_name_entry.delete(0,tk.END)
            for entry in entry_admin_widgets:
                entry.delete(0,tk.END)
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()  
    def delete_admin(self,frame):
       self.clear_frame(frame)
       user_name_label=tk.Label(frame,text="User _Id",font=("Times New Roman", 12))
       user_name_label.grid(row=0,column=0,padx=10,pady=10)
       user_name_entry=tk.Entry(frame,width=30)
       user_name_entry.grid(row=0,column=1,padx=10,pady=10)
       delete_user_button=tk.Button(frame,text="DELETE",font=("Times New Roman", 12),command=lambda:self.delete_User_check(user_name_entry))
       delete_user_button.grid(row=2,columnspan=3,padx=10,pady=10)
    def delete_User_check(self,user_name_entry):
       received_user_name = user_name_entry.get()
       if not received_user_name:
          messages="User Name can't be empty"
          simpledialog.messagebox.showinfo("error", messages)
          return
       try:
           mydb, cursor = self.establish_connection()
           query =  "DELETE FROM admin_table WHERE user_name = %s"
           cursor.execute(query, (received_user_name,))
           mydb.commit()

           if cursor.rowcount > 0:
              info="User  Has been removed Successfully"
              simpledialog.messagebox.showinfo("Delete Status",info)
              user_name_entry.delete(0, tk.END)
              user_name_entry.focus_set()
           else:
              info="Unable to Remove the User"
              simpledialog.messagebox.showinfo("Delete Status",info)
              user_name_entry.delete(0, tk.END)
              user_name_entry.focus_set()
       except mysql.connector.Error as error:
          simpledialog.messagebox.showinfo("Delete User Status", error)
          user_name_entry.delete(0, tk.END)
          user_name_entry.focus_set()
       finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close() 
    def create_new_student(self,frame):
        self.clear_frame(frame)
        std_id_label=tk.Label(frame,text="Student_Id",font=("Times New Roman", 12))
        std_id_label.grid(row=0,column=0,padx=10,pady=10)
        std_id=tk.Entry(frame,width=30)
        std_id.grid(row=0,column=1,padx=10,pady=10)
        std_name_label=tk.Label(frame,text="Sudent_Name",font=("Times New Roman", 12))
        std_name_label.grid(row=1,column=0,padx=10,pady=10)
        std_name=tk.Entry(frame,width=30)
        std_name.grid(row=1,column=1,padx=10,pady=10)
        branch_label=tk.Label(frame,text="Branch",font=("Times New Roman", 12))
        branch_label.grid(row=2,column=0,padx=10,pady=10)
        branch=tk.Entry(frame,width=20)
        branch.grid(row=2,column=1,padx=10,pady=10)
        year_label=tk.Label(frame,text="Year",font=("Times New Roman", 12))
        year_label.grid(row=3,column=0,padx=10,pady=10)
        year=tk.Entry(frame,width=20)
        year.grid(row=3,column=1,padx=10,pady=10)
        phone_number_label=tk.Label(frame,text="phone number",font=("Times New Roman", 12))
        phone_number_label.grid(row=4,column=0,padx=10,pady=10)
        phone_number=tk.Entry(frame,width=30)
        phone_number.grid(row=4,column=1,padx=10,pady=10)
        student_address_label=tk.Label(frame,text="Adress",font=("Times New Roman", 12))
        student_address_label.grid(row=5,column=0,padx=10,pady=10)
        student_address=tk.Entry(frame,width=30)
        student_address.grid(row=5,column=1,padx=10,pady=10)
        age_label=tk.Label(frame,text="age",font=("Times New Roman", 12))
        age_label.grid(row=6,column=0,padx=10,pady=10)
        age=tk.Entry(frame,width=30)
        age.grid(row=6,column=1,padx=10,pady=10)
        create_student=tk.Button(frame,text="Create_student",font=("Times New Roman", 12),command=lambda:self.Created_student_button(std_id,std_name,branch,year,phone_number,student_address,age))
        create_student.grid(row=8,columnspan=3,padx=10,pady=10)
    def Created_student_button(self,std_id,std_name,branch,year,phone_number,student_address,age):
        errors_created_student=[]
        if not (std_id.get() and std_name.get() and branch.get() and year.get(),phone_number.get(),student_address.get(),age.get()):
          errors_created_student.append("All fields are required.")
        else:
           if len(std_id.get()) != 10:
             errors_created_student.append("Student Id  must be 10 characters long.")
           if not std_name.get().isalpha():
             errors_created_student.append("Student Name must contain only alphabets.")
           if not branch.get().isalpha():
             errors_created_student.append("Branch  name must contain only alphabets.")
           if not year.get().isdigit():
             errors_created_student.append(" Student Year  must contain only integers.")
           if not phone_number.get().isdigit():
             errors_created_student.append(" Phone number  must contain only integers.")
           if len(phone_number.get()) != 10:
             errors_created_student.append("Phone Number  must be 10 characters long.")
           if not age.get().isdigit():
              errors_created_student.append("Age must contain only Numbers.")   
        if errors_created_student:
           tk.messagebox.showerror("Error", "\n".join(errors_created_student))
           std_id.delete(0, tk.END)
           std_name.delete(0, tk.END)
           branch.delete(0, tk.END)
           year.delete(0, tk.END)
           phone_number.delete(0, tk.END)
           student_address.delete(0, tk.END)
           age.delete(0, tk.END)
           std_id.focus_set() 
           return
        else:
          std_id_value=std_id.get()
          std_name_value =std_name.get()
          branch_value= branch.get()
          year_value=year.get()
          std_phone_number_value=phone_number.get()
          student_address_value=student_address.get()
          age_value= age.get()
          try:
            mydb, cursor = self.establish_connection()
            cursor = mydb.cursor()
            query = "INSERT INTO student_database (std_id, std_name, branch, year, phone_number, address, age) VALUES (%s, %s, %s, %s,%s,%s,%s)"
            cursor.execute(query, (std_id_value, std_name_value, branch_value, year_value,std_phone_number_value,student_address_value,age_value))
            mydb.commit()
            if cursor.rowcount > 0:
              info=f"Student Added successfully \n Student_Id ={std_id_value}\n Name={std_name_value}\n Branch={branch_value} \n Year={year_value}\n Phone Number={std_phone_number_value}\nAddress={student_address_value}\n Age ={age_value} "
              simpledialog.messagebox.showinfo("STUDENT Status", info)
              std_id.delete(0, tk.END)
              std_name.delete(0, tk.END)
              branch.delete(0, tk.END)
              year.delete(0, tk.END)
              phone_number.delete(0, tk.END)
              student_address.delete(0, tk.END)
              age.delete(0, tk.END)
              std_id.focus_set() 
            else:
               info="Unable to add the Student"
               std_id.delete(0, tk.END)
               std_name.delete(0, tk.END)
               branch.delete(0, tk.END)
               year.delete(0, tk.END)
               phone_number.delete(0, tk.END)
               student_address.delete(0, tk.END)
               age.delete(0, tk.END)
               std_id.focus_set()
          except mysql.connector.Error as error:
           simpledialog.messagebox.showinfo("STUDENT Status", error)
           std_id.delete(0, tk.END)
           std_name.delete(0, tk.END)
           branch.delete(0, tk.END)
           year.delete(0, tk.END)
           phone_number.delete(0, tk.END)
           student_address.delete(0, tk.END)
           age.delete(0, tk.END)
           std_id.focus_set()
          finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()  
    def update_student(self,frame):
      self.clear_frame(frame)
      var_std_name = tk.BooleanVar()
      name = tk.Checkbutton(frame, text="Student Name", variable=var_std_name)
      name.grid(row=2, column=1)
      var_branch=tk.BooleanVar()
      branch=tk.Checkbutton(frame,text="branch",variable=var_branch)
      branch.grid(row=4, column=1)
      var_year=tk.BooleanVar()
      year=tk.Checkbutton(frame,text="year",variable=var_year)
      year.grid(row=6, column=1)
      var_phone_number = tk.BooleanVar()
      phone_number = tk.Checkbutton(frame, text="phone_number", variable=var_phone_number)
      phone_number.grid(row=8, column=1)
      var_address=tk.BooleanVar()
      address=tk.Checkbutton(frame, text="address", variable=var_address)
      address.grid(row=10, column=1)
      var_age = tk.BooleanVar()
      age = tk.Checkbutton(frame, text="gender", variable=var_age)
      age.grid(row=12, column=1)
      button = tk.Button(frame, text="submit", command=lambda:self.call_student(frame,var_std_name,var_branch,var_year,var_phone_number,var_address,var_age))
      button.grid(row=16, column=1)
    def call_student(self,frame,var_std_name,var_branch,var_year,var_phone_number,var_address,var_age):
       self.disable_widgets(frame)
       self.checkbox_student_value(var_std_name,var_branch,var_year,var_phone_number,var_address,var_age,frame)
    def checkbox_student_value(self,var_std_name,var_branch,var_year,var_phone_number,var_address,var_age,frame):
       #std_id, std_name, branch, year, phone_number, address, age, num_books_borrowed
       values_student = []
       if var_std_name.get():
           values_student.append("std_name")
       if var_branch.get():
           values_student.append("branch")
       if var_year.get():
          values_student.append("year")
       if var_phone_number.get():
           values_student.append("phone_number")
       if var_address.get():
          values_student.append("address")
       if var_age.get():
          values_student.append("age")
       print("Selected values are:", values_student)
       self.creating_student_widgets(values_student,frame)
    def creating_student_widgets(self,values_student,frame):
       self.clear_frame(frame)
       row=1
       user_name=tk.Label(frame, text="std_id",font=("Times New Roman", 12))
       user_name.grid(row=0, column=0,padx=10,pady=10)
       std_id_entry = tk.Entry(frame, width=30)
       std_id_entry.grid(row=0, column=1,padx=10,pady=10)
       entry_student_widgets=[]
       for name in values_student:
        name_label=tk.Label(frame, text=name,font=("Times New Roman", 12))
        name_label.grid(row=row, column=0)
        entry=tk.Entry(frame, width=30)
        entry.grid(row=row, column=1,padx=10,pady=10)
        entry_student_widgets.append(entry)
        row += 1
       submit = tk.Button(frame, text=" Update ", command=lambda:self.update_student_query(values_student,entry_student_widgets, std_id_entry))
       submit.grid(row=row, columnspan=2,padx=10,pady=10)
    def update_student_query(self,values_student,entry_student_widgets, std_id_entry):
       received_std_id=std_id_entry.get()   
       errors_created_student=[]
       if not received_std_id:
          errors_created_student.append("Book_id Can't be Empty") 
       if len(std_id_entry.get()) != 10:
             errors_created_student.append("Student Id  must be 10 characters long.")    
       for entry in entry_student_widgets:
         if not entry.get():
            errors_created_student.append("Please fill in all the entry fields.")
       for name, entry in zip(values_student, entry_student_widgets):
           if name=="std_name":
             if not entry.get().isalpha():
                 errors_created_student.append("Student Name must contain only alphabets.")
           if name=="branch":
              if not entry.get().isalpha():
                errors_created_student.append("Branch  name must contain only alphabets.")
           if name=="year":  
              if not entry.get().isdigit():
                errors_created_student.append(" Student Year  must contain only integers.")
           if name=="phone_number":
              if not entry.get().isdigit():
                 errors_created_student.append(" Phone number  must contain only integers.")
              if len(entry.get()) != 10:
                 errors_created_student.append("Phone Number  must be 10 characters long.")
           if name=="age":
              if not entry.get().isdigit():
                   errors_created_student.append("Age must contain only Numbers.")   
       if errors_created_student:
           tk.messagebox.showerror("Error", "\n".join(errors_created_student))
           std_id_entry.delete(0, tk.END)
           for entry in received_std_id:
              entry.delete(0,tk.END)
       else: 
        set_student_clauses = []
        for value, entry in zip(values_student, entry_student_widgets):
          entry_value = entry.get()
          set_clause = f"{value}='{entry_value}'"  
          set_student_clauses.append(set_clause)
        set_values = ", ".join(set_student_clauses)
        try:
            mydb, cursor = self.establish_connection()
            cursor = mydb.cursor()
            query = f"UPDATE student_database SET {set_values} WHERE std_id = '{received_std_id}'"
            cursor.execute(query)
            mydb.commit()
            if cursor.rowcount > 0:
               student_info=f"Updated Student info: {set_values}"
               simpledialog.messagebox.showinfo("student Update Status", student_info)
               for entry in received_std_id:
                  entry.delete(0,tk.END)
               std_id_entry.delete(0, tk.END)

            else:
               student_info="unable to update the student info"
               simpledialog.messagebox.showinfo("student Update Status", student_info)
               std_id_entry.delete(0, tk.END)
               for entry in received_std_id:
                  entry.delete(0,tk.END)
        except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("STUDENT Status", error)
            student_info=f"Updated Student info: {set_values}"
            std_id_entry.delete(0, tk.END)
            for entry in received_std_id:
                  entry.delete(0,tk.END)
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()        
    def delete_student(self,frame):
       self.clear_frame(frame)
       std_id_label=tk.Label(frame,text="std_Id",font=("Times New Roman", 12))
       std_id_label.grid(row=0,column=0,padx=10,pady=10)
       std_id_entry=tk.Entry(frame,width=30)
       std_id_entry.grid(row=0,column=1,padx=10,pady=10)
       delete_user_button=tk.Button(frame,text="DELETE",font=("Times New Roman", 12),command=lambda:self.delete_student_check(std_id_entry))
       delete_user_button.grid(row=2,columnspan=3,padx=10,pady=10)
    def delete_student_check(self,std_id_entry):
       received_std_id_entry = std_id_entry.get()
       if not received_std_id_entry:
          messages="Student_Id can't be empty"
          simpledialog.messagebox.showinfo("error", messages)
          return
       try:
           mydb, cursor = self.establish_connection()
           query =  "DELETE FROM student_database WHERE std_id = %s"
           cursor.execute(query, (received_std_id_entry,))
           mydb.commit()

           if cursor.rowcount > 0:
              info="Student Has been removed Successfully"
              simpledialog.messagebox.showinfo("Delete Status",info)
              std_id_entry.delete(0, tk.END)
              std_id_entry.focus_set()
           else:
              info="Unable to Remove Student "
              simpledialog.messagebox.showinfo("Delete Status",info)
              std_id_entry.delete(0, tk.END)
              std_id_entry.focus_set()
       except mysql.connector.Error as error:
          simpledialog.messagebox.showinfo("Delete User Status", error)
          std_id_entry.delete(0, tk.END)
          std_id_entry.focus_set()
       finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close() 
    def get_student_history(self,frame):
        self.clear_frame(frame)
        student_user_id=tk.Label(frame,text="User_id",font=("Times New Roman", 12))
        student_user_id.grid(row=0,column=0,padx=10,pady=10)
        student_user_id_entry=tk.Entry(frame,width=30)
        student_user_id_entry.grid(row=0,column=1,padx=10,pady=10)
        student_login=tk.Button(frame,text="Get_History",font=("Times New Roman", 12),command=lambda:self.check_history(frame,student_user_id_entry))
        student_login.grid(row=4,columnspan=2,padx=10,pady=10)
    def check_history(self,frame,student_user_id_entry):
       if not student_user_id_entry.get():
          messages="student_user_id and Password can't be empty"
          simpledialog.messagebox.showinfo("error", messages)
          return
       user_name = student_user_id_entry.get()
       self.history(frame, user_name)
    def student_login(self):
        print("login")
        self.T.destroy()
        self.T4=tk.Tk()
        self.T4.title("Student login Page")
        student_frame=ttk.Frame(self.T4,borderwidth=6, relief="raised")
        student_frame.pack()
        student_user_id=tk.Label(student_frame,text="User_id",font=("Times New Roman", 12))
        student_user_id.grid(row=0,column=0,padx=10,pady=10)
        student_user_id_entry=tk.Entry(student_frame,width=30)
        student_user_id_entry.grid(row=0,column=1,padx=10,pady=10)
        student_password=tk.Label(student_frame,text="Password",font=("Times New Roman", 12))
        student_password.grid(row=1,column=0,padx=10,pady=10)
        student_password_entry=tk.Entry(student_frame,width=30,show="*")
        student_password_entry.grid(row=1,column=1,padx=10,pady=10)
        forgot_user_name=tk.Label(student_frame,text='forgot_user_name',font=("Times New Roman", 12))
        forgot_user_name.grid(row=2,column=0,padx=10,pady=10)
        forgot_user_name.bind("<Enter>", lambda event: forgot_user_name.config(font=("Times New Roman", 12, "underline")))
        forgot_user_name.bind("<Leave>", lambda event: forgot_user_name.config(font=("Times New Roman", 11)))
        forgot_user_name.bind("<Button-1>", self.forget_user_name)
        std_back_button=tk.Button(student_frame,text="Back to Home",font=("Times New Roman", 12),command=lambda:self.back_to_home(self.T4))
        std_back_button.grid(row=6,column=0,padx=10,pady=10,sticky="sw")
        forgot_password=tk.Label(student_frame,text='forgot_password',font=("Times New Roman", 12))
        forgot_password.grid(row=2,column=3,padx=10,pady=10)
        forgot_password.bind("<Enter>", lambda event: forgot_password.config(font=("Times New Roman", 12, "underline")))
        forgot_password.bind("<Leave>", lambda event: forgot_password.config(font=("Times New Roman", 11)))
        forgot_password.bind("<Button-1>", self.forget_password)
        student_login=tk.Button(student_frame,text="Login",font=("Times New Roman", 12),command=lambda:self.login_check(student_user_id_entry,student_password_entry))
        student_login.grid(row=4,columnspan=2,padx=10,pady=10)
        self.T4.mainloop()
    def login_check(self,student_user_id_entry,student_password_entry):
         if not(student_user_id_entry.get()and student_password_entry.get()):
           messages="student_user_id and Password can't be empty"
           simpledialog.messagebox.showinfo("error", messages)
           return
         user_name = student_user_id_entry.get()
         password = student_password_entry.get()
         try:
           mydb, cursor = self.establish_connection()
           query= "SELECT * FROM login_credentials WHERE login_id=%s AND password=%s"
           cursor.execute(query,(user_name,password))
           user =cursor.fetchone()
           if user: 
              student_user_id_entry.delete(0, tk.END)
              student_password_entry.delete(0, tk.END)
              self.T4.withdraw()
              self.student_page(user_name)
           else:
             simpledialog.messagebox.showinfo("Error", "Invalid credentials")
             student_user_id_entry.delete(0, tk.END)
             student_password_entry.delete(0, tk.END)
             student_user_id_entry.focus_set()
         except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("Login Status", error)
            student_user_id_entry.delete(0, tk.END)
            student_password_entry.delete(0, tk.END)
            student_user_id_entry.focus_set()    
         finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close() 
    def student_page(self,user_name):
        self.T5=tk.Toplevel()
        self.T5.title("student_page")
        menubar=tk.Menu(self.T5)
        menubar_options=tk.Menu(menubar,tearoff=0)
        menubar_options.add_command(label="History",command=lambda:self.history(frame,user_name))
        menubar_options.add_command(label="Chatbot",command=lambda:self.chatbot(frame))
        menubar_options.add_command(label="Change password",command=self.change_password)
        menubar_options.add_command(label="Log_out",command=lambda:self.logout(self.T5))
        menubar.add_cascade(label="Menu", menu=menubar_options)
        self.T5.config(menu=menubar)
        frame=ttk.Frame(self.T5,borderwidth=6, relief="raised")
        frame.pack()
        label=tk.Label(frame,text="Login successful",font=("Times New Roman", 12))
        label.grid(rowspan=5,columnspan=5,padx=20,pady=20)
        self.T5.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.T5.mainloop() 
    def history(self,frame,user_name):
       self.clear_frame(frame)
       print(user_name)
       try:
            mydb, cursor = self.establish_connection()
            query = """
                  SELECT i.issue_id, i.book_id ,b.book_name, b.book_author, b.book_published_year, s.std_name, s.branch,  s.year, s.phone_number, i.issue_date, i.return_date,i.return_on, i.fine
                  FROM issue_book i
                  JOIN books_database b ON i.book_id = b.book_id
                  JOIN student_database s ON i.student_id = s.std_id
                  WHERE i.student_id = %s
            """
            cursor.execute(query, (user_name,))
            rows = cursor.fetchall()
            if rows:
               history_text = scrolledtext.ScrolledText(frame, wrap=tk.WORD, width=60, height=20)
               history_text.pack(fill=tk.BOTH, expand=True)
               message = "Student History:\n"
               for row in rows:
                message += f"Issue ID: {row[0]}\n"
                message += f"Book ID: {row[1]}\n"
                message += f"Book Name: {row[2]}\n"
                message += f"Author: {row[3]}\n"
                message += f"Published Year: {row[4]}\n"
                message += f"Student Name: {row[5]}\n"
                message += f"Branch: {row[6]}\n"
                message += f"Year: {row[7]}\n"
                message += f"Student Phone Number: {row[8]}\n"
                message += f"Issue Date: {row[9]}\n"
                message += f"Return Date: {row[10]}\n"
                message += f"Returned on: {row[11]}\n"
                message += f"Fine: ${row[12]}\n\n"
               history_text.insert(tk.END, message)
               history_text.config(state="disabled")  # Disable editing 
            else:
               messagebox.showinfo("Student History", "No history found for the student.")
       except Exception as e:
          messagebox.showerror("Error", f"An error occurred: {e}")
       finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():
             cursor.close()   
    def chatbot(self, frame):
      self.clear_frame(frame)
      entry = tk.Entry(frame, width=30, fg="grey")
      entry.insert(0, "Message me here")
      entry.bind("<FocusIn>", lambda event: self.on_entry_focus(event, entry))
      entry.grid(row=1, column=1)
      text_widget = tk.Text(frame, width=40, height=10, state="disabled")
      text_widget.grid(row=0, columnspan=3, padx=10, pady=10)
      submit_button = tk.Button(frame, text="➔", font=("Arial", 12), cursor="hand2", command=lambda: self.submit_message(frame, entry, text_widget))
      submit_button.grid(row=1, column=2, padx=(5, 0))
    def submit_message(self, frame, entry, text_widget):
       message = entry.get()
       text_widget.config(state="normal") 
       text_widget.insert(tk.END, message + "\n")  
       text_widget.config(state="disabled") 
       entry.delete(0, tk.END)
      #  entry.insert(0, "Message me here")
       entry.config(fg="grey")
    def on_entry_focus(self, event, entry):
        if entry.get() == "Message me here":
            entry.delete(0, tk.END)    
    def forget_password(self,event=None):
       if hasattr(self, 'T2') and self.T2.winfo_exists():
         self.T2.withdraw()
       if hasattr(self, 'T4') and self.T4.winfo_exists():
         self.T4.withdraw()
       self.T7=tk.Tk()
       self.T7.title("forget_Passwords")
       forget_frame=tk.Frame(self.T7,borderwidth=6, relief="raised")
       forget_frame.pack()
       Back=tk.Label(forget_frame,text='forgot_user_name',font=("Times New Roman", 4))
       Back.grid(row=0,column=0,padx=10,pady=10)
       Back.bind("<Enter>", lambda event: Back.config(font=("Times New Roman", 12, "underline")))
       Back.bind("<Leave>", lambda event: Back.config(font=("Times New Roman", 11)))
       Back.bind("<Button-1>", lambda event: self.back(self.T7))
       username_stdid_label=tk.Label(forget_frame,text=" User_name/std_id ",font=("Times New Roman", 12))
       username_stdid_label.grid(row=0,column=1,padx=10,pady=10)
       username_stdid=tk.Entry(forget_frame,width=30)
       username_stdid.grid(row=0,column=2,padx=10,pady=10)
       phone_number_label=tk.Label(forget_frame,text=" Phone_Number ",font=("Times New Roman", 12))
       phone_number_label.grid(row=2,column=0,padx=10,pady=10)
       phone_number_entry=tk.Entry(forget_frame,width=30)
       phone_number_entry.grid(row=2,column=1,padx=10,pady=10)
       submit=tk.Button(forget_frame,text=" Submit ",font=("Times New Roman", 12),command=lambda:self.forget_password_process(username_stdid,phone_number_entry))
       submit.grid(row=4,column=1,padx=10,pady=10)
       self.T7.protocol("WM_DELETE_WINDOW", self.on_closing)
       self.T7.mainloop()
    def forget_password_process(self,username_stdid,phone_number_entry): 
       user_name_std_id = username_stdid.get()
       phone_number_entered = phone_number_entry.get()
       print("entered phone number =",phone_number_entered)
       print("user/id enter ",user_name_std_id)
       print("Data type of phone_number_entered:", type(phone_number_entered))
       if not (user_name_std_id and phone_number_entered):
         print("Please enter both username/student ID and phone number")
         # self.T7.deiconify()
         username_stdid.delete(0,tk.END)
         phone_number_entry.delete(0,tk.END)
         return
       error = []
       if not phone_number_entered.isdigit():
         error.append("Phone number should contain only digits")
       if len(phone_number_entered) != 10:
         error.append("Phone number should be 10 digits")
       if error:
         print(error)
         username_stdid.delete(0, tk.END)
         phone_number_entry.delete(0, tk.END)
         print("Please go back and correct the errors")
         # self.T7.deiconify()
         username_stdid.delete(0,tk.END)
         phone_number_entry.delete(0,tk.END)
         return
       else:
          mydb, cursor = self.establish_connection()
          query_admin = "SELECT phone_number FROM admin_table WHERE user_name = %s"
          cursor.execute(query_admin, (user_name_std_id,))
          result_admin = cursor.fetchone()
          query_student = "SELECT phone_number FROM student_database WHERE std_id = %s"
          cursor.execute(query_student, (user_name_std_id,))
          result_student = cursor.fetchone()
          if result_admin:
           phone_number = result_admin[0]
           print("Phone Number (from admin_table):", phone_number)
           print("Data type of phone_number:", type(phone_number))
           if str(phone_number) == phone_number_entered:
               if not phone_number_entered.startswith("+91"):
                   phone_number_entered = "+91" + phone_number_entered
               otp = self.generate_otp()
               self.send_otp(phone_number_entered, otp)
               if otp:
                  self.T7.destroy()  
                  self.enter_opt(otp,user_name_std_id)         
           else:
              info="Phone u entered for admin was in correct"
              simpledialog.messagebox.showinfo("STUDENT Status", info)
              username_stdid.delete(0,tk.END)
              phone_number_entry.delete(0,tk.END)
              self.T7.deiconify()    
          elif result_student:
           phone_number = result_student[0]
           print("Phone Number (from student_database):", phone_number)
           print("Data type of phone_number:", type(phone_number))
           if str(phone_number) == phone_number_entered:
                  if not phone_number_entered.startswith("+91"):
                   phone_number_entered = "+91" + phone_number_entered
                  otp = self.generate_otp()
                  print("Data type of otp:", type(otp))
                  self.send_otp(phone_number_entered, otp)
                  if otp:
                   self.T7.destroy()
                   self.enter_opt(otp, user_name_std_id)         
           else:
              info="Phone u entered for student was in correct"
              simpledialog.messagebox.showinfo("STUDENT Status", info)
              username_stdid.delete(0,tk.END)
              phone_number_entry.delete(0,tk.END)
              self.T7.deiconify()   
          else:
            info="No phone number found for the provided user_id / Std _id"
            simpledialog.messagebox.showinfo("STUDENT Status", info)
            mydb.close()
            username_stdid.delete(0,tk.END)
            phone_number_entry.delete(0,tk.END)
            self.T7.deiconify()        
    def enter_opt(self,otp,user_name_std_id):
       self.T8 = tk.Tk()
       self.T8.title("forget")
       frame_opt=tk.Frame(self.T8,borderwidth=6, relief="raised")
       frame_opt.pack() 
       entry=tk.Entry(frame_opt,width=40)
       entry.grid(row=0,columnspan=4,padx=10,pady=10)
       submit = tk.Button(frame_opt, text="submit opt", borderwidth=6, relief="raised", command=lambda:self.submit_opt(frame_opt,otp,entry,user_name_std_id))
       submit.grid(row=2, columnspan=4, padx=10, pady=10)
       Back=tk.Label(frame_opt,text='forgot_user_name',font=("Times New Roman", 4))
       Back.grid(row=0,column=0,padx=10,pady=10)
       Back.bind("<Enter>", lambda event: Back.config(font=("Times New Roman", 12, "underline")))
       Back.bind("<Leave>", lambda event: Back.config(font=("Times New Roman", 11)))
       Back.bind("<Button-1>", lambda event: self.back(self.T8))
       self.T8.protocol("WM_DELETE_WINDOW", self.on_closing)
       self.T8.mainloop()
       print(otp)
    def generate_otp(self):
        return str(random.randint(1000, 9999))
    def send_otp(self, to_number, otp):
        account_sid = 'ACc2784ae2f1756e3dab652f03e3826776'
        auth_token = 'c9e3c5a3b8206b25148fa6f9baa0883d'
        twilio_number = '+16202548140'
        client = Client(account_sid, auth_token)
        message = client.messages.create(
            body=f"Your OTP is: {otp}",
            from_=twilio_number,
            to=to_number
        )
        print("OTP sent successfully!")  
    def submit_opt(self,frame_opt,otp,entry,user_name_std_id):
       entered_opt=entry.get()
       print("Entered_opt",entered_opt)
       if entered_opt==otp:
          self.clear_frame(frame_opt)
          enter_password=tk.Label(frame_opt,text="Enter New password",font=("Times New Roman", 12))
          enter_password.grid(row=0,column=0,padx=10,pady=10)
          enter_new_password=tk.Entry(frame_opt,width=30)
          enter_new_password.grid(row=0,column=1,padx=10,pady=10)
          new_set_password_button=tk.Button(frame_opt,text="submit",borderwidth=6, relief="raised", command=lambda:self.new_set_password(enter_new_password,user_name_std_id))
          new_set_password_button.grid(row=2,columnspan=4,padx=10,pady=10)
       else:
          info="entered opt is incorrect"
          simpledialog.messagebox.showinfo("STUDENT Status", info)
          entry.delete(0,tk.END)
    def new_set_password(self,enter_new_password,user_name_std_id):
       new_password=enter_new_password.get()
       if not enter_new_password.get():
          info="password can't be empty"
          simpledialog.messagebox.showinfo("STUDENT Status", info)
          return
       try:
          mydb, cursor = self.establish_connection()
          query = "UPDATE login_credentials SET password = %s WHERE login_id = %s"
          cursor.execute(query, (new_password,user_name_std_id))
          mydb.commit()
          if cursor.rowcount >0:
             info= f"successfully changed password for the User_name / std_id = {user_name_std_id}"
             simpledialog.messagebox.showinfo("STUDENT Status", info)
             if hasattr(self, 'T8') and self.T8.winfo_exists():
                self.T8.destroy()
                print("T8 destroyed ")
             if hasattr(self, 'T2') and self.T2.winfo_exists():
                     self.T2.deiconify()
             if hasattr(self, 'T4') and self.T4.winfo_exists():
                  self.T4.deiconify()
             print("admin login page ")
          else:
             info="unable to change the pass word " 
             simpledialog.messagebox.showinfo("STUDENT Status", info)
       except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("SQL ERROR", error)
            enter_new_password.delete(0,tk.END)
       finally:
            if 'mydb' in locals():
              mydb.close()
            if 'cursor' in locals():  
              cursor.close()        
    def forget_user_name(self,event=None):
       if hasattr(self, 'T2') and self.T2.winfo_exists():
         self.T2.withdraw()
       if hasattr(self, 'T4') and self.T4.winfo_exists():
         self.T4.withdraw()
       self.T9=tk.Tk()
       self.T9.title("forget User name")
       username_frame=tk.Frame(self.T9,borderwidth=6, relief="raised")
       username_frame.pack()
       Back=tk.Label(username_frame,text='forgot_user_name',font=("Times New Roman", 10))
       Back.grid(row=0,column=0,padx=10,pady=10)
       Back.bind("<Enter>", lambda event: Back.config(font=("Times New Roman", 12, "underline")))
       Back.bind("<Leave>", lambda event: Back.config(font=("Times New Roman", 11)))
       Back.bind("<Button-1>", lambda event: self.back(self.T9))
       username_label=tk.Label(username_frame,text=" Admin Name / Student Name",font=("Times New Roman", 12))
       username_label.grid(row=0,column=1,padx=10,pady=10)
       username_entery=tk.Entry(username_frame,width=30)
       username_entery.grid(row=0,column=2,padx=10,pady=10)
       user_phonenumber=tk.Label(username_frame,text=" Admin Phonenumber / Student Phonenumber ",font=("Times New Roman", 12))
       user_phonenumber.grid(row=2,column=0,padx=10,pady=10)
       user_phonenumber_entery=tk.Entry(username_frame,width=30)
       user_phonenumber_entery.grid(row=2,column=1,padx=10,pady=10)
       get_username=tk.Button(username_frame,text=" Get User Name ",borderwidth=6, relief="raised",command=lambda:self.get_username(username_entery,user_phonenumber_entery))
       get_username.grid(row=4,columnspan=4 ,padx=10,pady=10)
       self.T9.protocol("WM_DELETE_WINDOW", self.on_closing)
       self.T9.mainloop()
    def get_username(self,username_entery,user_phonenumber_entery):  
       if not(username_entery.get() and user_phonenumber_entery.get()):
          info="All fields should be filled"
          simpledialog.messagebox.showinfo("STUDENT Status", info)
       error_user=[]
       if not re.match("^[a-zA-Z\s\W]+$", username_entery.get()):
          error_user.append("user Name can't have the Number ")
       if not user_phonenumber_entery.get().isdigit():
          error_user.append("Phone Number should only contain Number")
       if  len(user_phonenumber_entery.get()) !=10:
          error_user.append("Phone should be of 10 digits!")
       if error_user:
          tk.messagebox.showerror("Error", "\n".join(error_user))
          username_entery.delete(0,tk.END)
          user_phonenumber_entery.delete(0,tk.END)
          username_entery.focus_set()
          return
       else:
         name=username_entery.get()
         phone_number=user_phonenumber_entery.get()
         try:
            mydb, cursor = self.establish_connection()
            query_admin = "SELECT user_name FROM admin_table WHERE name = %s and phone_number = %s"
            cursor.execute(query_admin, (name,phone_number))
            result_admin = cursor.fetchone()
            query_student = "SELECT std_id FROM student_database WHERE std_name = %s and phone_number= %s"
            cursor.execute(query_student, (name,phone_number))
            result_student = cursor.fetchone()
            if result_admin:
                User_name = result_admin[0]
                simpledialog.messagebox.showinfo("User Name", User_name)
            elif result_student:
               std_id = result_student[0]
               simpledialog.messagebox.showinfo("Student ID", std_id)
            else:
               simpledialog.messagebox.showinfo("Information", "No matching record found.")
            if result_admin:
               if not phone_number.startswith("+91"):
                 phone_number = "+91" + phone_number
               self.send_otp(phone_number, User_name)
            elif result_student:
               if not phone_number.startswith("+91"):
                  phone_number = "+91" + phone_number
               self.send_otp(phone_number, std_id)
            if hasattr(self, 'T2') and self.T2.winfo_exists():
               self.T2.deiconify()
               self.T9.destroy()
            if hasattr(self, 'T4') and self.T4.winfo_exists():
               self.T4.deiconify()
               self.T9.destroy()
         except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("SQL ERROR", error)
            username_entery.delete(0,tk.END)
            user_phonenumber_entery.delete(0,tk.END)
            username_entery.focus_set()
         except twilio.base.exceptions.TwilioRestException as twilio_error:
            error_message = f"Twilio Error: {twilio_error}"
            simpledialog.messagebox.showinfo("Twilio Error", error_message)
            if hasattr(self, 'T2') and self.T2.winfo_exists():
               self.T2.deiconify()
               self.T9.destroy()
            if hasattr(self, 'T4') and self.T4.winfo_exists():
               self.T4.deiconify()
               self.T9.destroy()
         finally:
            if 'mydb' in locals():
              mydb.close()
            if 'cursor' in locals():  
              cursor.close()       
    def change_password(self):
       if hasattr(self, 'T3') and self.T3.winfo_exists():
                  self.T3.destroy()
       if hasattr(self, 'T5') and self.T5.winfo_exists():
                  self.T5.destroy()
       self.T6=tk.Tk()
       self.T6.title("Change_password")
       changepassword=ttk.Frame(self.T6,borderwidth=6, relief="raised")
       changepassword.pack()
       user_name=tk.Label(changepassword,text="User Name",font=("Times New Roman", 12))
       user_name.grid(row=0,column=0,padx=10,pady=10)
       user_name_entry=tk.Entry(changepassword,width=30)
       user_name_entry.grid(row=0,column=1,padx=10,pady=10)
       old_password_label=tk.Label(changepassword,text="old_Password",font=("Times New Roman", 12))
       old_password_label.grid(row=2,column=0,padx=10,pady=10)
       old_password=tk.Entry(changepassword,width=30,show="*")
       old_password.grid(row=2,column=1,padx=10,pady=10)
       new_password_label=tk.Label(changepassword,text="new_password",font=("Times New Roman", 12))
       new_password_label.grid(row=3,column=0,padx=10,pady=10)
       new_password=tk.Entry(changepassword,width=30,show="*")
       new_password.grid(row=3,column=1,padx=10,pady=10)
       login_button=tk.Button(changepassword,text="changepassword",font=("Times New Roman", 12),command=lambda: self.making_passwordchange(user_name_entry,old_password,new_password,T6))
       login_button.grid(row=5,columnspan=4,padx=10,pady=10)
       self.T9.protocol("WM_DELETE_WINDOW", self.p_u_on_closing)
       self.T6.mainloop()  
    def making_passwordchange(self,user_name_entry,old_password,new_password,T6):
        if not(user_name_entry.get()and old_password.get(),new_password.get()):
           messages="Any Filed  Can't be Empty"
           simpledialog.messagebox.showinfo("error", messages)
           return
        user_name_value = user_name_entry.get()
        old_password_value = old_password.get()
        new_password_value = new_password.get()
        try:
           mydb, cursor = self.establish_connection()
           query= "UPDATE login_credentials SET password = %s WHERE login_id = %s AND password = %s"
           cursor.execute(query,(new_password_value,user_name_value,old_password_value))
           mydb.commit()
           user_name_entry.delete(0, tk.END)
           old_password.delete(0, tk.END)
           new_password.delete(0, tk.END)
           if cursor.rowcount > 0:
            simpledialog.messagebox.showinfo("Password Change", "Password changed successfully")
            T6.destroy()
            if hasattr(self, 'T2') and self.T2.winfo_exists():
                  self.T2.deiconify()
            if hasattr(self, 'T4') and self.T4.winfo_exists():
                  self.T4.deiconify()
           else:
            simpledialog.messagebox.showinfo("Password Change", "Incorrect username or old password")
            user_name_entry.delete(0, tk.END)
            old_password.delete(0, tk.END)
            new_password.delete(0, tk.END)
            user_name_entry.focus_set()
        except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("Password Change", error)
            T6.destroy()
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()  
    def clear_frame(self,frame):
      for widget in frame.winfo_children():
         widget.destroy()
    def establish_connection(self):
       try:
          mydb = mysql.connector.connect(
             host="localhost",
             user="root",
             password="Hari@25082002",
             database="library_database")
          cursor = mydb.cursor()
          return mydb, cursor
       except mysql.connector.Error as error:
          print("Error while connecting to MySQL", error)
          raise
    def logout(self, window):
       confirm_logout = messagebox.askquestion("Confirm Logout", "Are you sure you want to logout?")
       if confirm_logout == 'yes':
             window.destroy() 
             if hasattr(self, 'T3') and window == self.T3:
               self.T2.deiconify()
             elif hasattr(self, 'T5') and window == self.T5:
                self.T4.deiconify()
       else:
          pass
    def back_to_home(self,window):
       confirm_logout = messagebox.askquestion("Confirm Logout", "Are you sure you want to logout?")
       if confirm_logout == 'yes':
          window.destroy()
          new_smart_library_instance = SmartLibrary()
       else:
          pass
    def disable_widgets(self,frame):
       for widget in frame.winfo_children():
          widget.config(state="disabled")
    def on_closing(self):
       pass
    def back(self,window,event=None):
      if hasattr(self, 'T2') and self.T2.winfo_exists():
         self.T2.destroy()
      if hasattr(self, 'T4') and self.T4.winfo_exists():
         self.T4.destroy()
      if window.winfo_exists():
         window.destroy()
      self.__init__()
      
library = SmartLibrary()



Admin_page
B192300001


In [ ]:
import mysql.connector
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox,simpledialog
import datetime
import re
from tkinter import scrolledtext
from twilio.rest import Client
import random 
import twilio


class SmartLibrary:   
    def __init__(self):
        self.T = tk.Tk()
        self.T.title('Home_page')
        self.Home = ttk.Frame(self.T ,borderwidth=6, relief="raised")
        self.Home.pack()
        self.Heading = tk.Label(self.Home, text='Smart Library', font=("Times New Roman", 18, 'bold'))
        self.Heading.grid(row=0, columnspan=3)
        self.admin = tk.Button(self.Home, text="Admin", font=("Times New Roman", 12), command=self.admin_login)
        self.admin.grid(row=1, column=3, padx=20, pady=20)
        self.student = tk.Button(self.Home, text="Student", font=("Times New Roman", 12), command=self.student_login)
        self.student.grid(row=4, column=3, padx=20, pady=20)
        self.T.mainloop()    
    def admin_login(self):
        self.T.destroy()
        self.T2=tk.Tk()
        self.T2.title("admin_login")
        front_page=ttk.Frame(self.T2,borderwidth=6, relief="raised")
        front_page.pack()
        user_name=tk.Label(front_page,text="User Name",font=("Times New Roman", 12))
        user_name.grid(row=0,column=0,padx=10,pady=10)
        user_name_entry=tk.Entry(front_page,width=30)
        user_name_entry.grid(row=0,column=1,padx=10,pady=10)
        password=tk.Label(front_page,text="Password",font=("Times New Roman", 12))
        password.grid(row=2,column=0,padx=10,pady=10)
        password_entry=tk.Entry(front_page,width=30,show="*")
        password_entry.grid(row=2,column=1,padx=10,pady=10)
        forgot_user_name=tk.Label(front_page,text='forgot_user_name',font=("Times New Roman", 12))
        forgot_user_name.grid(row=3,column=0,padx=10,pady=10)
        forgot_user_name.bind("<Enter>", lambda event: forgot_user_name.config(font=("Times New Roman", 12, "underline")))
        forgot_user_name.bind("<Leave>", lambda event: forgot_user_name.config(font=("Times New Roman", 11)))
        forgot_user_name.bind("<Button-1>", self.forget_user_name)
        back_button=tk.Button(front_page,text="Back to Home",font=("Times New Roman", 12),command=lambda:self.back_to_home(self.T2))
        back_button.grid(row=6,column=0,padx=10,pady=10,sticky="sw")
        forgot_password=tk.Label(front_page,text='forgot_password',font=("Times New Roman", 12))
        forgot_password.grid(row=3,column=1,padx=10,pady=10)
        forgot_password.bind("<Enter>", lambda event: forgot_password.config(font=("Times New Roman", 12, "underline")))
        forgot_password.bind("<Leave>", lambda event: forgot_password.config(font=("Times New Roman", 11)))
        forgot_password.bind("<Button-1>", self.forget_password)
        login_button=tk.Button(front_page,text="login",font=("Times New Roman", 12),command=lambda: self.check(user_name_entry, password_entry))
        login_button.grid(row=4,columnspan=4,padx=10,pady=10)
        self.T2.mainloop()
    def check(self,user_name_entry,password_entry):
        if not(user_name_entry.get()and password_entry.get()):
           messages="empty"
           simpledialog.messagebox.showinfo("error", messages)
           return
        user_name = user_name_entry.get()
        password = password_entry.get()
        try:
           mydb, cursor = self.establish_connection()
           query= "SELECT * FROM login_credentials WHERE login_id=%s AND password=%s"
           cursor.execute(query,(user_name,password))
           user =cursor.fetchone()
           if user:
            user_name_entry.delete(0, tk.END)
            password_entry.delete(0, tk.END)
            self.T2.withdraw()
            self.admin_page()
           else:
            simpledialog.messagebox.showinfo("Error", "no found")
            user_name_entry.delete(0, tk.END)
            password_entry.delete(0, tk.END)
            user_name_entry.focus_set()
        except mysql.connector.Error as err:
            print("Something went wrong: {}".format(err)) 
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()
    def admin_page(self):
        print("Admin_page")
        self.T3 = tk.Toplevel()
        self.T3.title("Admin_page")
        menubar=tk.Menu(self.T3)
        menubar_options=tk.Menu(menubar,tearoff=0)
        menubar_options.add_command(label="Issue books",command=lambda:self.issue_books(frame))
        menubar_options.add_command(label="Return books",command=lambda:self.return_books(frame))
        books_submenu=tk.Menu(menubar_options,tearoff=0)
        books_submenu.add_command(label="Add Book",command=lambda:self.add_books(frame))
        books_submenu.add_command(label="Update Book",command=lambda:self.update_books(frame))
        books_submenu.add_command(label="Delete/Remove Book",command=lambda:self.delete_book(frame))
        menubar_options.add_cascade(label="Books", menu=books_submenu)
        admin_submenu=tk.Menu(menubar,tearoff=0)
        admin_submenu.add_command(label="create_admin",command=lambda:self.create_new_admin(frame))
        admin_submenu.add_command(label="Update_admininfo",command=lambda:self.update_admin(frame))
        admin_submenu.add_command(label="Delete/Remove admin",command=lambda:self.delete_admin(frame))
        menubar_options.add_cascade(label="ADMIN",menu=admin_submenu)
        student_submenu=tk.Menu(menubar,tearoff=0)
        student_submenu.add_command(label="Create New Student",command=lambda:self.create_new_student(frame))
        student_submenu.add_command(label="Update StudentInfo",command=lambda:self.update_student(frame))
        student_submenu.add_command(label="Delete/Remove Student",command=lambda:self.delete_student(frame))
        student_submenu.add_command(label="Student History",command=lambda:self.get_student_history(frame))
        menubar_options.add_cascade(label="STUDENT",menu=student_submenu)
        menubar_options.add_command(label="Change password",command=self.change_password)
        menubar_options.add_command(label="Log_out",command=lambda:self.logout(self.T3))
        menubar.add_cascade(label="Menu", menu=menubar_options)
        self.T3.config(menu=menubar)
        frame=ttk.Frame(self.T3,borderwidth=6, relief="raised")
        frame.pack()
        label=tk.Label(frame,text="Login successful",font=("Times New Roman", 12))
        label.grid(rowspan=5,columnspan=5,padx=20,pady=20)
        self.T3.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.T3.mainloop()
    def issue_books(self,frame):
        self.clear_frame(frame)
        student_id=tk.Label(frame,text="Student_id",font=("Times New Roman", 12))
        student_id.grid(row=0,column=0,padx=10,pady=10)
        student_id_entry=tk.Entry(frame,width=30)
        student_id_entry.grid(row=0,column=1,padx=10,pady=10)
        book_id=tk.Label(frame,text="Book_Id",font=("Times New Roman", 12))
        book_id.grid(row=1,column=0,padx=10,pady=10)
        book_id_entry=tk.Entry(frame,width=30)
        book_id_entry.grid(row=1,column=1,padx=10,pady=10)
        Submit=tk.Button(frame,text="Issuse",font=("Times New Roman", 12),command=lambda:self.issue(student_id_entry, book_id_entry))
        Submit.grid(row=3,columnspan=2,padx=10,pady=10)
    def issue(self, student_id_entry, book_id_entry):
      errors = []
      if not (student_id_entry.get() and book_id_entry.get()):
          errors.append("All fields are required.")
      else:
       if len(student_id_entry.get()) != 10:
          errors.append("Student ID must be 10 characters long.")
       if not book_id_entry.get().isdigit():
          errors.append("Book ID must contain only integers.")
      
      if errors:
          tk.messagebox.showerror("Error", "\n".join(errors))
          student_id_entry.delete(0, tk.END)
          book_id_entry.delete(0, tk.END)
          student_id_entry.focus_set()
          return
      else:
       student_id_value=student_id_entry.get()
       book_id_value=book_id_entry.get()
       today = datetime.date.today()
       today_formatted = today.strftime("%d-%m-%Y")
       return_date = today + datetime.timedelta(days=15)
       return_date_formatted = return_date.strftime("%d-%m-%Y")
       try:
          mydb, cursor = self.establish_connection()
          cursor.execute("SELECT num_books_borrowed FROM student_database WHERE std_id = %s", (student_id_value,))
          num_books_borrowed = cursor.fetchone()[0]
          if num_books_borrowed >= 3:
             tk.messagebox.showinfo("Book Limit Exceeded", "The student has already borrowed the maximum number of books.")
             student_id_entry.delete(0, tk.END)
             book_id_entry.delete(0, tk.END)
             student_id_entry.focus_set()
             return
          cursor.execute("SELECT status FROM books_database WHERE book_id = %s", (book_id_value,))
          book_status = cursor.fetchone()
          if not book_status or book_status[0] != 'available':
             tk.messagebox.showinfo("Book Status", "The book is not available for issuing.")
             student_id_entry.delete(0, tk.END)
             book_id_entry.delete(0, tk.END)
             student_id_entry.focus_set()
             return
          cursor.execute("UPDATE books_database SET status = 'borrowed' WHERE book_id = %s", (book_id_value,))
          new_num_books_borrowed = num_books_borrowed + 1
          cursor.execute("UPDATE student_database SET num_books_borrowed = %s WHERE std_id = %s", (new_num_books_borrowed, student_id_value))
          query = "INSERT INTO issue_book ( student_id, book_id, issue_date, return_date) VALUES (%s, %s, %s, %s)"
          cursor.execute(query, ( student_id_value, book_id_value, today,return_date))
          mydb.commit()
          if cursor.rowcount > 0:
             info_add_book=f"Book issused  successfully \n Student_id ={student_id_value}\n Book _id={book_id_value}\n issue_Date={today_formatted}\n Return_Date ={return_date_formatted}"
             simpledialog.messagebox.showinfo("Book Issued", info_add_book)
             student_id_entry.delete(0, tk.END)
             book_id_entry.delete(0, tk.END)
             student_id_entry.focus_set()
          else:
             simpledialog.messagebox.showinfo("Update_info", "Books was not issued")
             student_id_entry.delete(0, tk.END)
             book_id_entry.delete(0, tk.END)
             student_id_entry.focus_set() 
       except mysql.connector.Error as error:  
          simpledialog.messagebox.showinfo("Book status Status", error)
          student_id_entry.delete(0, tk.END)
          book_id_entry.delete(0, tk.END)
          student_id_entry.focus_set() 
       finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()     
    def return_books(self,frame):
        self.clear_frame(frame)
        student_id=tk.Label(frame,text="Student_id",font=("Times New Roman", 12))
        student_id.grid(row=0,column=0,padx=10,pady=10)
        student_id_entry=tk.Entry(frame,width=30)
        student_id_entry.grid(row=0,column=1,padx=10,pady=10)
        book_id=tk.Label(frame,text="Book_Id",font=("Times New Roman", 12))
        book_id.grid(row=1,column=0,padx=10,pady=10)
        book_id_entry=tk.Entry(frame,width=30)
        book_id_entry.grid(row=1,column=1,padx=10,pady=10)
        Submit=tk.Button(frame,text="Return",font=("Times New Roman", 12),command=lambda:self.return_book_check(student_id_entry, book_id_entry))
        Submit.grid(row=3,columnspan=2,padx=10,pady=10)
    def return_book_check(self,student_id_entry, book_id_entry):
      errors = []
      if not (student_id_entry.get() and book_id_entry.get()):
          errors.append("All fields are required.")
      else:
       if len(student_id_entry.get()) != 10:
          errors.append("Student ID must be 10 characters long.")
       if not book_id_entry.get().isdigit():
          errors.append("Book ID must contain only integers.")
      if errors:
          tk.messagebox.showerror("Error", "\n".join(errors))
          student_id_entry.delete(0, tk.END)
          book_id_entry.delete(0, tk.END)
          student_id_entry.focus_set()
          return
      else:
       student_id_value=student_id_entry.get()
       book_id_value=book_id_entry.get()
       today = datetime.date.today()
       try:
          mydb, cursor = self.establish_connection()
          cursor.execute("SELECT student_id FROM issue_book WHERE book_id = %s", (book_id_entry.get(),))
          original_borrower_id = cursor.fetchone()[0]
          if original_borrower_id != student_id_entry.get():
            messagebox.showerror("Error", "Only the original borrower can return the book.")
            student_id_entry.delete(0, tk.END)
            book_id_entry.delete(0, tk.END)
            student_id_entry.focus_set()
            return
          cursor.execute("UPDATE books_database SET status = 'available' WHERE book_id = %s", (book_id_value,))
          cursor.execute("SELECT num_books_borrowed FROM student_database WHERE std_id = %s", (student_id_value,))
          num_books_borrowed = cursor.fetchone()[0]
          new_num_books_borrowed = max(num_books_borrowed - 1,0)
          cursor.execute("UPDATE student_database SET num_books_borrowed = %s WHERE std_id = %s", (new_num_books_borrowed, student_id_value))
          cursor.execute("SELECT return_date FROM issue_book WHERE student_id = %s",(student_id_value,))
          return_date=cursor.fetchone()[0]
          if today>return_date:
             print(return_date)
             print(today)
             difference = (today - return_date).days
             fine = max(difference * 2, 0) 
             cursor.execute("UPDATE issue_book SET fine = %s,return_on= %s WHERE student_id = %s", (fine,today, student_id_value))
             fine_message = f"Fine: ${fine}"
             print(fine)
          else:
             fine_message = "No fine"
             cursor.execute("UPDATE issue_book SET return_on= %s WHERE student_id = %s", (today, student_id_value))
          mydb.commit()
          if cursor.rowcount > 0:
             simpledialog.messagebox.showinfo("Return Status",fine_message)
             student_id_entry.delete(0, tk.END)
             book_id_entry.delete(0, tk.END)
             student_id_entry.focus_set()
          else:
             simpledialog.messagebox.showinfo("Return_status", "Already returned")
             student_id_entry.delete(0, tk.END)
             book_id_entry.delete(0, tk.END)
             student_id_entry.focus_set() 
       except mysql.connector.Error as error: 
          simpledialog.messagebox.showinfo("Book status Status", error)
          student_id_entry.delete(0, tk.END)
          book_id_entry.delete(0, tk.END)
          student_id_entry.focus_set() 
       finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()    
    def add_books(self,frame):
        self.clear_frame(frame)
        book_id_label=tk.Label(frame,text="Book Id",font=("Times New Roman", 12))
        book_id_label.grid(row=0,column=0,padx=10, pady=10)
        book_id=tk.Entry(frame,width=30)
        book_id.grid(row=0,column=1,padx=10,pady=10)
        book_name_label=tk.Label(frame,text="Book Name",font=("Times New Roman", 12))
        book_name_label.grid(row=1,column=0,padx=10,pady=10)
        book_name=tk.Entry(frame,width=30)
        book_name.grid(row=1,column=1,padx=10,pady=10)
        book_author_label=tk.Label(frame,text="Author_Name",font=("Times New Roman", 12))
        book_author_label.grid(row=2,column=0,padx=10,pady=10)
        book_author=tk.Entry(frame,width=30)
        book_author.grid(row=2,column=1,padx=10,pady=10)
        book_published_year_label=tk.Label(frame,text="Published Year",font=("Times New Roman", 12))
        book_published_year_label.grid(row=3,column=0,padx=10,pady=10)
        book_published_year=tk.Entry(frame,width=20)
        book_published_year.grid(row=3,column=1,padx=10,pady=10)
        add_book=tk.Button(frame,text="Add Book",font=("Times New Roman", 12),command=lambda:self.add_book_button(book_id,book_name,book_author,book_published_year))
        add_book.grid(row=5,columnspan=2,padx=10,pady=10)  
    def add_book_button(self,book_id,book_name,book_author,book_published_year):
        errors_add_book=[]
        if not (book_id.get() and book_name.get() and book_author.get() and book_published_year.get()):
          errors_add_book.append("All fields are required.")
        else:
          if not book_id.get().isdigit():
           errors_add_book.append("Book id  must contain only integers.")
          if not re.match("^[a-zA-Z\s\d\W]+$", book_name.get()):
            errors_add_book.append("Book title name must contain only alphabets.")
          if not re.match("^[a-zA-Z\s\d\W]+$", book_author.get()):
            errors_add_book.append("Book Author name must contain only alphabets.")
          if not book_published_year.get().isdigit():
           errors_add_book.append("Year must contain only integers.")
        if errors_add_book:
           tk.messagebox.showerror("Error", "\n".join(errors_add_book))
           book_id.delete(0, tk.END)
           book_name.delete(0, tk.END)
           book_author.delete(0, tk.END)
           book_published_year.delete(0, tk.END)
           book_id.focus_set()
           return
        else:
         book_id_val=book_id.get()
         book_name_val=book_name.get()
         book_author_val=book_author.get()
         book_published_year_val=book_published_year.get()
         try:
            mydb, cursor = self.establish_connection()
            cursor = mydb.cursor()
            query = "INSERT INTO books_database (book_id, book_name, book_author, book_published_year) VALUES (%s, %s, %s, %s)"
            cursor.execute(query, (book_id_val, book_name_val, book_author_val, book_published_year_val))
            mydb.commit()
            if cursor.rowcount > 0:
             info_add_book=f"Book Added  successfully \n Book Id ={book_id_val}\n Book Name={book_name_val}\n Book Author={book_author_val}\n Book Published Year ={book_published_year_val}"
             simpledialog.messagebox.showinfo("Book Issued", info_add_book)
             book_id.delete(0, tk.END)
             book_name.delete(0, tk.END)
             book_author.delete(0, tk.END)
             book_published_year.delete(0, tk.END)
             book_id.focus_set()
            else:
             simpledialog.messagebox.showinfo("Update_info", "Books was not added")
             book_id.delete(0, tk.END)
             book_name.delete(0, tk.END)
             book_author.delete(0, tk.END)
             book_published_year.delete(0, tk.END)
             book_id.focus_set()
         except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("Book status Status", error)
            book_id.delete(0, tk.END)
            book_name.delete(0, tk.END)
            book_author.delete(0, tk.END)
            book_published_year.delete(0, tk.END)
            book_id.focus_set()  
         finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()     
    def update_books(self,frame):
      self.clear_frame(frame)
      var_book_name = tk.BooleanVar()
      book_name = tk.Checkbutton(frame, text="book_name", variable=var_book_name)
      book_name.grid(row=2, column=1)
      var_book_author = tk.BooleanVar()
      book_author = tk.Checkbutton(frame, text="book_author", variable=var_book_author)
      book_author.grid(row=4, column=1)
      var_book_published_year = tk.BooleanVar()
      book_published_year = tk.Checkbutton(frame, text="book_published_year", variable=var_book_published_year)
      book_published_year.grid(row=6, column=1)
      button = tk.Button(frame, text="submit", command=lambda:self.book_call(frame,var_book_name,var_book_author,var_book_published_year))
      button.grid(row=8, column=1)
    def book_call(self,frame,var_book_name,var_book_author,var_book_published_year):
       self.disable_widgets(frame)
       self.checkbox_value(var_book_name,var_book_author,var_book_published_year,frame)
    def checkbox_value(self,var_book_name,var_book_author,var_book_published_year,frame):
       values = []
       #book_id, book_name, book_author, book_published_year, status
       if var_book_name.get():
           values.append("book_name")
       if var_book_author.get():
           values.append("book_author")
       if var_book_published_year.get():
           values.append("book_published_year")
       print("Selected values are:", values)
       self.creating_widgets(values,frame)
    def creating_widgets(self,values,frame):
       self.clear_frame(frame)
       row=1
       book_id=tk.Label(frame, text="Book_id",font=("Times New Roman", 12))
       book_id.grid(row=0, column=0,padx=10,pady=10)
       book_id_entry = tk.Entry(frame, width=30)
       book_id_entry.grid(row=0, column=1,padx=10,pady=10)
       entry_widgets=[]
       for name in values:
        name_label=tk.Label(frame, text=name,font=("Times New Roman", 12))
        name_label.grid(row=row, column=0)
        entry=tk.Entry(frame, width=30)
        entry.grid(row=row, column=1,padx=10,pady=10)
        entry_widgets.append(entry)
        row += 1
       submit = tk.Button(frame, text=" Update ", command=lambda:self.update_query(values,entry_widgets, book_id_entry))
       submit.grid(row=row, columnspan=2,padx=10,pady=10)
    def update_query(self,values, entry_widgets,book_id_entry):
       received_book_id=book_id_entry.get()
       #book_id, book_name, book_author, book_published_year, status
       errors_add_book=[]
       for entry in entry_widgets:
         if not entry.get():
            errors_add_book.append("Please fill in all the entry fields.")
       if not received_book_id:
          errors_add_book.append("Book_id Can't be Empty")  
       for name,entry in zip(values, entry_widgets):
        if name=="book_name":
            if not re.match("^[a-zA-Z\s\W]+$", entry.get()):
                errors_add_book.append("Book title name must contain only alphabets.")
        if name=="book_author":
              if not re.match("^[a-zA-Z\s\W]+$", entry.get()):
                errors_add_book.append("Book Author name must contain only alphabets.")
        if name=="book_published_year":
             if not entry.get().isdigit():
                  errors_add_book.append("Year must contain only integers.")
       if errors_add_book:
          tk.messagebox.showerror("Error", "\n".join(errors_add_book)) 
          for entry in entry_widgets:
            entry.delete(0,tk.END)
          book_id_entry.delete(0,tk.END)
       else: 
        set_clauses = []
        for value, entry in zip(values, entry_widgets):
          entry_value = entry.get()
          set_clause = f"{value}='{entry_value}'"  
          set_clauses.append(set_clause)
        set_values = ", ".join(set_clauses)
        try:
            mydb, cursor = self.establish_connection()
            cursor = mydb.cursor()
            query = f"UPDATE books_database SET {set_values} WHERE book_id = '{received_book_id}'"
            cursor.execute(query)
            mydb.commit()
            if cursor.rowcount > 0:
              info=f"Updated Book details:{set_values}"
              simpledialog.messagebox.showinfo("STUDENT Status", info)
              for entry in entry_widgets:
                 entry.delete(0,tk.END)
              book_id_entry.delete(0,tk.END)
            else:
              info="Unable to update the books data"
              simpledialog.messagebox.showinfo("STUDENT Status", info)
              for entry in entry_widgets:
                 entry.delete(0,tk.END)
              book_id_entry.delete(0,tk.END)     
        except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("STUDENT Status", error)
            for entry in entry_widgets:
                 entry.delete(0,tk.END)
            book_id_entry.delete(0,tk.END)
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()  
    def delete_book(self,frame):
       self.clear_frame(frame)
       book_id_label=tk.Label(frame,text="Book ID",font=("Times New Roman", 12))
       book_id_label.grid(row=0,column=0,padx=10,pady=10)
       book_id_entry=tk.Entry(frame,width=30)
       book_id_entry.grid(row=0,column=1,padx=10,pady=10)
       delete_button=tk.Button(frame,text="DELETE",font=("Times New Roman", 12),command=lambda:self.delete_book_check(book_id_entry))
       delete_button.grid(row=2,columnspan=3,padx=10,pady=10)
    def delete_book_check(self,book_id_entry):
        received_book_id = book_id_entry.get()
        if not received_book_id:
           messages="Book id cant be empty"
           simpledialog.messagebox.showinfo("error", messages)
           return
        try:
           mydb, cursor = self.establish_connection()
           query = "DELETE FROM books_database WHERE book_id = %s"
           cursor.execute(query, (received_book_id,))
           mydb.commit()
           if cursor.rowcount > 0:
              info="Record/Book Has been deleted/removed Successfully"
              simpledialog.messagebox.showinfo("Delete Status",info)
              book_id_entry.delete(0, tk.END)
              book_id_entry.focus_set()
           else:
              info="Unable to Delete the Record"
              simpledialog.messagebox.showinfo("Delete Status",info)
              book_id_entry.delete(0, tk.END)
              book_id_entry.focus_set()
        except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("Delete Book Status", error)
            book_id_entry.delete(0, tk.END)
            book_id_entry.focus_set()
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()  
    def create_new_admin(self,frame):
        self.clear_frame(frame)
        name_label=tk.Label(frame,text="Name",font=("Times New Roman", 12))
        name_label.grid(row=0,column=0,padx=10,pady=10)
        name=tk.Entry(frame,width=30)
        name.grid(row=0,column=1,padx=10,pady=10)
        user_name_label=tk.Label(frame,text="User Name",font=("Times New Roman", 12))
        user_name_label.grid(row=1,column=0,padx=10,pady=10)
        user_name=tk.Entry(frame,width=30)
        user_name.grid(row=1,column=1,padx=10,pady=10)
        phone_number_label=tk.Label(frame,text="Phone Number",font=("Times New Roman", 12))
        phone_number_label.grid(row=2,column=0,padx=10,pady=10)
        phone_number=tk.Entry(frame,width=30)
        phone_number.grid(row=2,column=1,padx=10,pady=10)
        gender_label=tk.Label(frame,text="gender",font=("Times New Roman", 12))
        gender_label.grid(row=3,column=0,padx=10,pady=10)
        gender=tk.Entry(frame,width=30)
        gender.grid(row=3,column=1,padx=10,pady=10)
        address_label=tk.Label(frame,text="Addres",font=("Times New Roman", 12))
        address_label.grid(row=4,column=0,padx=10,pady=10)
        address=tk.Entry(frame,width=30)
        address.grid(row=4,column=1,padx=10,pady=10)
        creat_admin=tk.Button(frame,text="Create admin",font=("Times New Roman", 12),command=lambda:self.create_admin_button(name,user_name,phone_number,gender,address))
        creat_admin.grid(row=6,columnspan=2,padx=10,pady=10)
    def create_admin_button(self,name,user_name,phone_number,gender,address):
        error_create_admin=[]
        if not (name.get() and user_name.get() and phone_number.get() and gender.get(),address.get()):
          error_create_admin.append("All fields are required.")
        else:
           if not name.get().isalpha():
             error_create_admin.append("Name name must contain only alphabets.")
           if not phone_number.get().isdigit():
             error_create_admin.append("Phone Number must contain only integers.")
           if len(phone_number.get()) != 10:
             error_create_admin.append("Phone Number  must be 10 characters long.")
           if not gender.get().isalpha():
              error_create_admin.append("gender must be only male or female")
        if error_create_admin:
           tk.messagebox.showerror("Error", "\n".join(error_create_admin))
           name.delete(0, tk.END)
           user_name.delete(0, tk.END)
           phone_number.delete(0, tk.END)
           gender.delete(0,tk.END)
           address.delete(0, tk.END)
           name.focus_set()
           return
        else:
           name_value=name.get()
           user_name_value=user_name.get()
           phone_number_value=phone_number.get()
           gender_value=gender.get()
           address_value=address.get()
        try:
            mydb, cursor = self.establish_connection()
            cursor = mydb.cursor()
            query = "INSERT INTO admin_table (name, user_name, phone_number, gender,address) VALUES (%s, %s, %s, %s,%s)"
            cursor.execute(query, (name_value, user_name_value, phone_number_value, gender_value,address_value))
            mydb.commit()
            if cursor.rowcount > 0:
             info=f"Admin Added successfully \n Name ={name_value}\n User_Name={user_name_value}\n Phone Number={phone_number_value}\n gender ={gender_value} \n Address={address_value}"
             simpledialog.messagebox.showinfo("Admin Status", info)
             name.delete(0, tk.END)
             user_name.delete(0, tk.END)
             phone_number.delete(0, tk.END)
             gender.delete(0, tk.END)
             address.delete(0, tk.END)
             name.focus_set()
            else:
               info="Unable to add the Admin"
               name.delete(0, tk.END)
               user_name.delete(0, tk.END)
               phone_number.delete(0, tk.END)
               gender.delete(0, tk.END)
               address.delete(0, tk.END)
               name.focus_set()
        except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("Admin Status", error)
            name.delete(0, tk.END)
            user_name.delete(0, tk.END)
            phone_number.delete(0, tk.END)
            gender.delete(0, tk.END)
            address.delete(0, tk.END)
            name.focus_set()
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()  
    def update_admin(self,frame):
      self.clear_frame(frame)
      var_name = tk.BooleanVar()
      name = tk.Checkbutton(frame, text="name", variable=var_name)
      name.grid(row=2, column=1)
      var_phone_number = tk.BooleanVar()
      phone_number = tk.Checkbutton(frame, text="phone_number", variable=var_phone_number)
      phone_number.grid(row=4, column=1)
      var_address=tk.BooleanVar()
      address=tk.Checkbutton(frame, text="address", variable=var_address)
      address.grid(row=6, column=1)
      var_gender = tk.BooleanVar()
      gender = tk.Checkbutton(frame, text="gender", variable=var_gender)
      gender.grid(row=8, column=1)
      button = tk.Button(frame, text="submit", command=lambda:self.admin_call(frame,var_name,var_phone_number,var_address,var_gender))
      button.grid(row=10, column=1)
    def admin_call(self,frame,var_name,var_phone_number,var_address,var_gender):
       self.disable_widgets(frame)
       self.checkbox_admin_value(var_name,var_phone_number,var_address,var_gender,frame)
    def checkbox_admin_value(self,var_name,var_phone_number,var_address,var_gender,frame):
       #name, user_name, phone_number, address, gender
       values_admin = []
       if var_name.get():
           values_admin.append("name")
       if var_phone_number.get():
           values_admin.append("phone_number")
       if var_address.get():
          values_admin.append("address")
       if var_gender.get():
           values_admin.append("gender")
       self.creating_admin_widgets(values_admin,frame)
    def creating_admin_widgets(self,values_admin,frame):
       self.clear_frame(frame)
       row=1
       user_name=tk.Label(frame, text="user_nmae",font=("Times New Roman", 12))
       user_name.grid(row=0, column=0,padx=10,pady=10)
       user_name_entry = tk.Entry(frame, width=30)
       user_name_entry.grid(row=0, column=1,padx=10,pady=10)
       entry_admin_widgets=[]
       for name in values_admin:
        name_label=tk.Label(frame, text=name,font=("Times New Roman", 12))
        name_label.grid(row=row, column=0)
        entry=tk.Entry(frame, width=30)
        entry.grid(row=row, column=1,padx=10,pady=10)
        entry_admin_widgets.append(entry)
        row += 1
       submit = tk.Button(frame, text=" Update ", command=lambda:self.update_admin_query(values_admin,entry_admin_widgets, user_name_entry))
       submit.grid(row=row, columnspan=2,padx=10,pady=10)
    def update_admin_query(self,values_admin, entry_admin_widgets,user_name_entry):
       received_user_name=user_name_entry.get()
              #name, user_name, phone_number, address, gender
       admin_errors=[]
       if not received_user_name:
          admin_errors.append("user_name Can't be Empty")
       for entry in entry_admin_widgets:
         if not entry.get():
            admin_errors.append("Please fill in all the entry fields.")
       for name,entry in zip(values_admin, entry_admin_widgets):
          if name=='name':
             if not re.match("^[a-zA-Z\s\W]+$", entry.get()):
                admin_errors.append("Name  must contain only alphabets")
          if name=="phone_number":
             if not entry.get().isdigit():
                admin_errors.append("Phone Number must contain only integers.")
             if len(entry.get()) != 10:
                admin_errors.append("Phone Number  must be 10 characters long.")    
          if name=="gender":
             if not entry.get().isalpha():
                admin_errors.append("gender must be only male or female") 
       if admin_errors:
          tk.messagebox.showerror("Error", "\n".join(admin_errors))
          user_name_entry.delete(0,tk.END)
          for entry in entry_admin_widgets:
             entry.delete(0,tk.END)
       else: 
        set_admin_clauses = []
        for value, entry in zip(values_admin, entry_admin_widgets):
          entry_value = entry.get()
          set_clause = f"{value}='{entry_value}'"  
          set_admin_clauses.append(set_clause)
        set_values = ", ".join(set_admin_clauses)
        try:
            mydb, cursor = self.establish_connection()
            cursor = mydb.cursor()
            query = f"UPDATE admin_table SET {set_values} WHERE user_name = '{received_user_name}'"
            cursor.execute(query)
            mydb.commit()
            if cursor.rowcount > 0:
             info_admin=f"Updated Admin info \n {set_values} "
             simpledialog.messagebox.showinfo("Admin Update status ", info_admin)
             user_name_entry.delete(0,tk.END)
             for entry in entry_admin_widgets:
                entry.delete(0,tk.END)
            else:
             info_admin="unable to Update the admin info"
             simpledialog.messagebox.showinfo("Admin Update status ", info_admin)
             user_name_entry.delete(0,tk.END)
             for entry in entry_admin_widgets:
                entry.delete(0,tk.END)
        except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("STUDENT Status", error)
            user_name_entry.delete(0,tk.END)
            for entry in entry_admin_widgets:
                entry.delete(0,tk.END)
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()  
    def delete_admin(self,frame):
       self.clear_frame(frame)
       user_name_label=tk.Label(frame,text="User _Id",font=("Times New Roman", 12))
       user_name_label.grid(row=0,column=0,padx=10,pady=10)
       user_name_entry=tk.Entry(frame,width=30)
       user_name_entry.grid(row=0,column=1,padx=10,pady=10)
       delete_user_button=tk.Button(frame,text="DELETE",font=("Times New Roman", 12),command=lambda:self.delete_User_check(user_name_entry))
       delete_user_button.grid(row=2,columnspan=3,padx=10,pady=10)
    def delete_User_check(self,user_name_entry):
       received_user_name = user_name_entry.get()
       if not received_user_name:
          messages="User Name can't be empty"
          simpledialog.messagebox.showinfo("error", messages)
          return
       try:
           mydb, cursor = self.establish_connection()
           query =  "DELETE FROM admin_table WHERE user_name = %s"
           cursor.execute(query, (received_user_name,))
           mydb.commit()

           if cursor.rowcount > 0:
              info="User  Has been removed Successfully"
              simpledialog.messagebox.showinfo("Delete Status",info)
              user_name_entry.delete(0, tk.END)
              user_name_entry.focus_set()
           else:
              info="Unable to Remove the User"
              simpledialog.messagebox.showinfo("Delete Status",info)
              user_name_entry.delete(0, tk.END)
              user_name_entry.focus_set()
       except mysql.connector.Error as error:
          simpledialog.messagebox.showinfo("Delete User Status", error)
          user_name_entry.delete(0, tk.END)
          user_name_entry.focus_set()
       finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close() 
    def create_new_student(self,frame):
        self.clear_frame(frame)
        std_id_label=tk.Label(frame,text="Student_Id",font=("Times New Roman", 12))
        std_id_label.grid(row=0,column=0,padx=10,pady=10)
        std_id=tk.Entry(frame,width=30)
        std_id.grid(row=0,column=1,padx=10,pady=10)
        std_name_label=tk.Label(frame,text="Sudent_Name",font=("Times New Roman", 12))
        std_name_label.grid(row=1,column=0,padx=10,pady=10)
        std_name=tk.Entry(frame,width=30)
        std_name.grid(row=1,column=1,padx=10,pady=10)
        branch_label=tk.Label(frame,text="Branch",font=("Times New Roman", 12))
        branch_label.grid(row=2,column=0,padx=10,pady=10)
        branch=tk.Entry(frame,width=20)
        branch.grid(row=2,column=1,padx=10,pady=10)
        year_label=tk.Label(frame,text="Year",font=("Times New Roman", 12))
        year_label.grid(row=3,column=0,padx=10,pady=10)
        year=tk.Entry(frame,width=20)
        year.grid(row=3,column=1,padx=10,pady=10)
        phone_number_label=tk.Label(frame,text="phone number",font=("Times New Roman", 12))
        phone_number_label.grid(row=4,column=0,padx=10,pady=10)
        phone_number=tk.Entry(frame,width=30)
        phone_number.grid(row=4,column=1,padx=10,pady=10)
        student_address_label=tk.Label(frame,text="Adress",font=("Times New Roman", 12))
        student_address_label.grid(row=5,column=0,padx=10,pady=10)
        student_address=tk.Entry(frame,width=30)
        student_address.grid(row=5,column=1,padx=10,pady=10)
        age_label=tk.Label(frame,text="age",font=("Times New Roman", 12))
        age_label.grid(row=6,column=0,padx=10,pady=10)
        age=tk.Entry(frame,width=30)
        age.grid(row=6,column=1,padx=10,pady=10)
        create_student=tk.Button(frame,text="Create_student",font=("Times New Roman", 12),command=lambda:self.Created_student_button(std_id,std_name,branch,year,phone_number,student_address,age))
        create_student.grid(row=8,columnspan=3,padx=10,pady=10)
    def Created_student_button(self,std_id,std_name,branch,year,phone_number,student_address,age):
        errors_created_student=[]
        if not (std_id.get() and std_name.get() and branch.get() and year.get(),phone_number.get(),student_address.get(),age.get()):
          errors_created_student.append("All fields are required.")
        else:
           if len(std_id.get()) != 10:
             errors_created_student.append("Student Id  must be 10 characters long.")
           if not std_name.get().isalpha():
             errors_created_student.append("Student Name must contain only alphabets.")
           if not branch.get().isalpha():
             errors_created_student.append("Branch  name must contain only alphabets.")
           if not year.get().isdigit():
             errors_created_student.append(" Student Year  must contain only integers.")
           if not phone_number.get().isdigit():
             errors_created_student.append(" Phone number  must contain only integers.")
           if len(phone_number.get()) != 10:
             errors_created_student.append("Phone Number  must be 10 characters long.")
           if not age.get().isdigit():
              errors_created_student.append("Age must contain only Numbers.")   
        if errors_created_student:
           tk.messagebox.showerror("Error", "\n".join(errors_created_student))
           std_id.delete(0, tk.END)
           std_name.delete(0, tk.END)
           branch.delete(0, tk.END)
           year.delete(0, tk.END)
           phone_number.delete(0, tk.END)
           student_address.delete(0, tk.END)
           age.delete(0, tk.END)
           std_id.focus_set() 
           return
        else:
          std_id_value=std_id.get()
          std_name_value =std_name.get()
          branch_value= branch.get()
          year_value=year.get()
          std_phone_number_value=phone_number.get()
          student_address_value=student_address.get()
          age_value= age.get()
          try:
            mydb, cursor = self.establish_connection()
            cursor = mydb.cursor()
            query = "INSERT INTO student_database (std_id, std_name, branch, year, phone_number, address, age) VALUES (%s, %s, %s, %s,%s,%s,%s)"
            cursor.execute(query, (std_id_value, std_name_value, branch_value, year_value,std_phone_number_value,student_address_value,age_value))
            mydb.commit()
            if cursor.rowcount > 0:
              info=f"Student Added successfully \n Student_Id ={std_id_value}\n Name={std_name_value}\n Branch={branch_value} \n Year={year_value}\n Phone Number={std_phone_number_value}\nAddress={student_address_value}\n Age ={age_value} "
              simpledialog.messagebox.showinfo("STUDENT Status", info)
              std_id.delete(0, tk.END)
              std_name.delete(0, tk.END)
              branch.delete(0, tk.END)
              year.delete(0, tk.END)
              phone_number.delete(0, tk.END)
              student_address.delete(0, tk.END)
              age.delete(0, tk.END)
              std_id.focus_set() 
            else:
               info="Unable to add the Student"
               std_id.delete(0, tk.END)
               std_name.delete(0, tk.END)
               branch.delete(0, tk.END)
               year.delete(0, tk.END)
               phone_number.delete(0, tk.END)
               student_address.delete(0, tk.END)
               age.delete(0, tk.END)
               std_id.focus_set()
          except mysql.connector.Error as error:
           simpledialog.messagebox.showinfo("STUDENT Status", error)
           std_id.delete(0, tk.END)
           std_name.delete(0, tk.END)
           branch.delete(0, tk.END)
           year.delete(0, tk.END)
           phone_number.delete(0, tk.END)
           student_address.delete(0, tk.END)
           age.delete(0, tk.END)
           std_id.focus_set()
          finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()  
    def update_student(self,frame):
      self.clear_frame(frame)
      var_std_name = tk.BooleanVar()
      name = tk.Checkbutton(frame, text="Student Name", variable=var_std_name)
      name.grid(row=2, column=1)
      var_branch=tk.BooleanVar()
      branch=tk.Checkbutton(frame,text="branch",variable=var_branch)
      branch.grid(row=4, column=1)
      var_year=tk.BooleanVar()
      year=tk.Checkbutton(frame,text="year",variable=var_year)
      year.grid(row=6, column=1)
      var_phone_number = tk.BooleanVar()
      phone_number = tk.Checkbutton(frame, text="phone_number", variable=var_phone_number)
      phone_number.grid(row=8, column=1)
      var_address=tk.BooleanVar()
      address=tk.Checkbutton(frame, text="address", variable=var_address)
      address.grid(row=10, column=1)
      var_age = tk.BooleanVar()
      age = tk.Checkbutton(frame, text="gender", variable=var_age)
      age.grid(row=12, column=1)
      button = tk.Button(frame, text="submit", command=lambda:self.call_student(frame,var_std_name,var_branch,var_year,var_phone_number,var_address,var_age))
      button.grid(row=16, column=1)
    def call_student(self,frame,var_std_name,var_branch,var_year,var_phone_number,var_address,var_age):
       self.disable_widgets(frame)
       self.checkbox_student_value(var_std_name,var_branch,var_year,var_phone_number,var_address,var_age,frame)
    def checkbox_student_value(self,var_std_name,var_branch,var_year,var_phone_number,var_address,var_age,frame):
       #std_id, std_name, branch, year, phone_number, address, age, num_books_borrowed
       values_student = []
       if var_std_name.get():
           values_student.append("std_name")
       if var_branch.get():
           values_student.append("branch")
       if var_year.get():
          values_student.append("year")
       if var_phone_number.get():
           values_student.append("phone_number")
       if var_address.get():
          values_student.append("address")
       if var_age.get():
          values_student.append("age")
       print("Selected values are:", values_student)
       self.creating_student_widgets(values_student,frame)
    def creating_student_widgets(self,values_student,frame):
       self.clear_frame(frame)
       row=1
       user_name=tk.Label(frame, text="std_id",font=("Times New Roman", 12))
       user_name.grid(row=0, column=0,padx=10,pady=10)
       std_id_entry = tk.Entry(frame, width=30)
       std_id_entry.grid(row=0, column=1,padx=10,pady=10)
       entry_student_widgets=[]
       for name in values_student:
        name_label=tk.Label(frame, text=name,font=("Times New Roman", 12))
        name_label.grid(row=row, column=0)
        entry=tk.Entry(frame, width=30)
        entry.grid(row=row, column=1,padx=10,pady=10)
        entry_student_widgets.append(entry)
        row += 1
       submit = tk.Button(frame, text=" Update ", command=lambda:self.update_student_query(values_student,entry_student_widgets, std_id_entry))
       submit.grid(row=row, columnspan=2,padx=10,pady=10)
    def update_student_query(self,values_student,entry_student_widgets, std_id_entry):
       received_std_id=std_id_entry.get()   
       errors_created_student=[]
       if not received_std_id:
          errors_created_student.append("Book_id Can't be Empty") 
       if len(std_id_entry.get()) != 10:
             errors_created_student.append("Student Id  must be 10 characters long.")    
       for entry in entry_student_widgets:
         if not entry.get():
            errors_created_student.append("Please fill in all the entry fields.")
       for name, entry in zip(values_student, entry_student_widgets):
           if name=="std_name":
             if not entry.get().isalpha():
                 errors_created_student.append("Student Name must contain only alphabets.")
           if name=="branch":
              if not entry.get().isalpha():
                errors_created_student.append("Branch  name must contain only alphabets.")
           if name=="year":  
              if not entry.get().isdigit():
                errors_created_student.append(" Student Year  must contain only integers.")
           if name=="phone_number":
              if not entry.get().isdigit():
                 errors_created_student.append(" Phone number  must contain only integers.")
              if len(entry.get()) != 10:
                 errors_created_student.append("Phone Number  must be 10 characters long.")
           if name=="age":
              if not entry.get().isdigit():
                   errors_created_student.append("Age must contain only Numbers.")   
       if errors_created_student:
           tk.messagebox.showerror("Error", "\n".join(errors_created_student))
           std_id_entry.delete(0, tk.END)
           for entry in received_std_id:
              entry.delete(0,tk.END)
       else: 
        set_student_clauses = []
        for value, entry in zip(values_student, entry_student_widgets):
          entry_value = entry.get()
          set_clause = f"{value}='{entry_value}'"  
          set_student_clauses.append(set_clause)
        set_values = ", ".join(set_student_clauses)
        try:
            mydb, cursor = self.establish_connection()
            cursor = mydb.cursor()
            query = f"UPDATE student_database SET {set_values} WHERE std_id = '{received_std_id}'"
            cursor.execute(query)
            mydb.commit()
            if cursor.rowcount > 0:
               student_info=f"Updated Student info: {set_values}"
               simpledialog.messagebox.showinfo("student Update Status", student_info)
               for entry in received_std_id:
                  entry.delete(0,tk.END)
               std_id_entry.delete(0, tk.END)

            else:
               student_info="unable to update the student info"
               simpledialog.messagebox.showinfo("student Update Status", student_info)
               std_id_entry.delete(0, tk.END)
               for entry in received_std_id:
                  entry.delete(0,tk.END)
        except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("STUDENT Status", error)
            student_info=f"Updated Student info: {set_values}"
            std_id_entry.delete(0, tk.END)
            for entry in received_std_id:
                  entry.delete(0,tk.END)
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()        
    def delete_student(self,frame):
       self.clear_frame(frame)
       std_id_label=tk.Label(frame,text="std_Id",font=("Times New Roman", 12))
       std_id_label.grid(row=0,column=0,padx=10,pady=10)
       std_id_entry=tk.Entry(frame,width=30)
       std_id_entry.grid(row=0,column=1,padx=10,pady=10)
       delete_user_button=tk.Button(frame,text="DELETE",font=("Times New Roman", 12),command=lambda:self.delete_student_check(std_id_entry))
       delete_user_button.grid(row=2,columnspan=3,padx=10,pady=10)
    def delete_student_check(self,std_id_entry):
       received_std_id_entry = std_id_entry.get()
       if not received_std_id_entry:
          messages="Student_Id can't be empty"
          simpledialog.messagebox.showinfo("error", messages)
          return
       try:
           mydb, cursor = self.establish_connection()
           query =  "DELETE FROM student_database WHERE std_id = %s"
           cursor.execute(query, (received_std_id_entry,))
           mydb.commit()

           if cursor.rowcount > 0:
              info="Student Has been removed Successfully"
              simpledialog.messagebox.showinfo("Delete Status",info)
              std_id_entry.delete(0, tk.END)
              std_id_entry.focus_set()
           else:
              info="Unable to Remove Student "
              simpledialog.messagebox.showinfo("Delete Status",info)
              std_id_entry.delete(0, tk.END)
              std_id_entry.focus_set()
       except mysql.connector.Error as error:
          simpledialog.messagebox.showinfo("Delete User Status", error)
          std_id_entry.delete(0, tk.END)
          std_id_entry.focus_set()
       finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close() 
    def get_student_history(self,frame):
        self.clear_frame(frame)
        student_user_id=tk.Label(frame,text="User_id",font=("Times New Roman", 12))
        student_user_id.grid(row=0,column=0,padx=10,pady=10)
        student_user_id_entry=tk.Entry(frame,width=30)
        student_user_id_entry.grid(row=0,column=1,padx=10,pady=10)
        student_login=tk.Button(frame,text="Get_History",font=("Times New Roman", 12),command=lambda:self.check_history(frame,student_user_id_entry))
        student_login.grid(row=4,columnspan=2,padx=10,pady=10)
    def check_history(self,frame,student_user_id_entry):
       if not student_user_id_entry.get():
          messages="student_user_id and Password can't be empty"
          simpledialog.messagebox.showinfo("error", messages)
          return
       user_name = student_user_id_entry.get()
       self.history(frame, user_name)
    def student_login(self):
        print("login")
        self.T.destroy()
        self.T4=tk.Tk()
        self.T4.title("Student login Page")
        student_frame=ttk.Frame(self.T4,borderwidth=6, relief="raised")
        student_frame.pack()
        student_user_id=tk.Label(student_frame,text="User_id",font=("Times New Roman", 12))
        student_user_id.grid(row=0,column=0,padx=10,pady=10)
        student_user_id_entry=tk.Entry(student_frame,width=30)
        student_user_id_entry.grid(row=0,column=1,padx=10,pady=10)
        student_password=tk.Label(student_frame,text="Password",font=("Times New Roman", 12))
        student_password.grid(row=1,column=0,padx=10,pady=10)
        student_password_entry=tk.Entry(student_frame,width=30,show="*")
        student_password_entry.grid(row=1,column=1,padx=10,pady=10)
        forgot_user_name=tk.Label(student_frame,text='forgot_user_name',font=("Times New Roman", 12))
        forgot_user_name.grid(row=2,column=0,padx=10,pady=10)
        forgot_user_name.bind("<Enter>", lambda event: forgot_user_name.config(font=("Times New Roman", 12, "underline")))
        forgot_user_name.bind("<Leave>", lambda event: forgot_user_name.config(font=("Times New Roman", 11)))
        forgot_user_name.bind("<Button-1>", self.forget_user_name)
        std_back_button=tk.Button(student_frame,text="Back to Home",font=("Times New Roman", 12),command=lambda:self.back_to_home(self.T4))
        std_back_button.grid(row=6,column=0,padx=10,pady=10,sticky="sw")
        forgot_password=tk.Label(student_frame,text='forgot_password',font=("Times New Roman", 12))
        forgot_password.grid(row=2,column=3,padx=10,pady=10)
        forgot_password.bind("<Enter>", lambda event: forgot_password.config(font=("Times New Roman", 12, "underline")))
        forgot_password.bind("<Leave>", lambda event: forgot_password.config(font=("Times New Roman", 11)))
        forgot_password.bind("<Button-1>", self.forget_password)
        student_login=tk.Button(student_frame,text="Login",font=("Times New Roman", 12),command=lambda:self.login_check(student_user_id_entry,student_password_entry))
        student_login.grid(row=4,columnspan=2,padx=10,pady=10)
        self.T4.mainloop()
    def login_check(self,student_user_id_entry,student_password_entry):
         if not(student_user_id_entry.get()and student_password_entry.get()):
           messages="student_user_id and Password can't be empty"
           simpledialog.messagebox.showinfo("error", messages)
           return
         user_name = student_user_id_entry.get()
         password = student_password_entry.get()
         try:
           mydb, cursor = self.establish_connection()
           query= "SELECT * FROM login_credentials WHERE login_id=%s AND password=%s"
           cursor.execute(query,(user_name,password))
           user =cursor.fetchone()
           if user: 
              student_user_id_entry.delete(0, tk.END)
              student_password_entry.delete(0, tk.END)
              self.T4.withdraw()
              self.student_page(user_name)
           else:
             simpledialog.messagebox.showinfo("Error", "Invalid credentials")
             student_user_id_entry.delete(0, tk.END)
             student_password_entry.delete(0, tk.END)
             student_user_id_entry.focus_set()
         except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("Login Status", error)
            student_user_id_entry.delete(0, tk.END)
            student_password_entry.delete(0, tk.END)
            student_user_id_entry.focus_set()    
         finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close() 
    def student_page(self,user_name):
        self.T5=tk.Toplevel()
        self.T5.title("student_page")
        menubar=tk.Menu(self.T5)
        menubar_options=tk.Menu(menubar,tearoff=0)
        menubar_options.add_command(label="History",command=lambda:self.history(frame,user_name))
        menubar_options.add_command(label="Chatbot",command=lambda:self.chatbot(frame))
        menubar_options.add_command(label="Change password",command=self.change_password)
        menubar_options.add_command(label="Log_out",command=lambda:self.logout(self.T5))
        menubar.add_cascade(label="Menu", menu=menubar_options)
        self.T5.config(menu=menubar)
        frame=ttk.Frame(self.T5,borderwidth=6, relief="raised")
        frame.pack()
        label=tk.Label(frame,text="Login successful",font=("Times New Roman", 12))
        label.grid(rowspan=5,columnspan=5,padx=20,pady=20)
        self.T5.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.T5.mainloop() 
    def history(self,frame,user_name):
       self.clear_frame(frame)
       print(user_name)
       try:
            mydb, cursor = self.establish_connection()
            query = """
                  SELECT i.issue_id, i.book_id ,b.book_name, b.book_author, b.book_published_year, s.std_name, s.branch,  s.year, s.phone_number, i.issue_date, i.return_date,i.return_on, i.fine
                  FROM issue_book i
                  JOIN books_database b ON i.book_id = b.book_id
                  JOIN student_database s ON i.student_id = s.std_id
                  WHERE i.student_id = %s
            """
            cursor.execute(query, (user_name,))
            rows = cursor.fetchall()
            if rows:
               history_text = scrolledtext.ScrolledText(frame, wrap=tk.WORD, width=60, height=20)
               history_text.pack(fill=tk.BOTH, expand=True)
               message = "Student History:\n"
               for row in rows:
                message += f"Issue ID: {row[0]}\n"
                message += f"Book ID: {row[1]}\n"
                message += f"Book Name: {row[2]}\n"
                message += f"Author: {row[3]}\n"
                message += f"Published Year: {row[4]}\n"
                message += f"Student Name: {row[5]}\n"
                message += f"Branch: {row[6]}\n"
                message += f"Year: {row[7]}\n"
                message += f"Student Phone Number: {row[8]}\n"
                message += f"Issue Date: {row[9]}\n"
                message += f"Return Date: {row[10]}\n"
                message += f"Returned on: {row[11]}\n"
                message += f"Fine: ${row[12]}\n\n"
               history_text.insert(tk.END, message)
               history_text.config(state="disabled")  # Disable editing 
            else:
               messagebox.showinfo("Student History", "No history found for the student.")
       except Exception as e:
          messagebox.showerror("Error", f"An error occurred: {e}")
       finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():
             cursor.close()   
    def chatbot(self, frame):
      self.clear_frame(frame)
      entry = tk.Entry(frame, width=30, fg="grey")
      entry.insert(0, "Message me here")
      entry.bind("<FocusIn>", lambda event: self.on_entry_focus(event, entry))
      entry.grid(row=1, column=1)
      text_widget = tk.Text(frame, width=40, height=10, state="disabled")
      text_widget.grid(row=0, columnspan=3, padx=10, pady=10)
      submit_button = tk.Button(frame, text="➔", font=("Arial", 12), cursor="hand2", command=lambda: self.submit_message(frame, entry, text_widget))
      submit_button.grid(row=1, column=2, padx=(5, 0))
    def submit_message(self, frame, entry, text_widget):
       message = entry.get()
       text_widget.config(state="normal") 
       text_widget.insert(tk.END, message + "\n")  
       text_widget.config(state="disabled") 
       entry.delete(0, tk.END)
      #  entry.insert(0, "Message me here")
       entry.config(fg="grey")
    def on_entry_focus(self, event, entry):
        if entry.get() == "Message me here":
            entry.delete(0, tk.END)    
    def forget_password(self,event=None):
       if hasattr(self, 'T2') and self.T2.winfo_exists():
         self.T2.withdraw()
       if hasattr(self, 'T4') and self.T4.winfo_exists():
         self.T4.withdraw()
       self.T7=tk.Tk()
       self.T7.title("forget_Passwords")
       forget_frame=tk.Frame(self.T7,borderwidth=6, relief="raised")
       forget_frame.pack()
       Back=tk.Label(forget_frame,text='forgot_user_name',font=("Times New Roman", 4))
       Back.grid(row=0,column=0,padx=10,pady=10)
       Back.bind("<Enter>", lambda event: Back.config(font=("Times New Roman", 12, "underline")))
       Back.bind("<Leave>", lambda event: Back.config(font=("Times New Roman", 11)))
       Back.bind("<Button-1>", lambda event: self.back(self.T7))
       username_stdid_label=tk.Label(forget_frame,text=" User_name/std_id ",font=("Times New Roman", 12))
       username_stdid_label.grid(row=0,column=1,padx=10,pady=10)
       username_stdid=tk.Entry(forget_frame,width=30)
       username_stdid.grid(row=0,column=2,padx=10,pady=10)
       phone_number_label=tk.Label(forget_frame,text=" Phone_Number ",font=("Times New Roman", 12))
       phone_number_label.grid(row=2,column=0,padx=10,pady=10)
       phone_number_entry=tk.Entry(forget_frame,width=30)
       phone_number_entry.grid(row=2,column=1,padx=10,pady=10)
       submit=tk.Button(forget_frame,text=" Submit ",font=("Times New Roman", 12),command=lambda:self.forget_password_process(username_stdid,phone_number_entry))
       submit.grid(row=4,column=1,padx=10,pady=10)
       self.T7.protocol("WM_DELETE_WINDOW", self.on_closing)
       self.T7.mainloop()
    def forget_password_process(self,username_stdid,phone_number_entry): 
       user_name_std_id = username_stdid.get()
       phone_number_entered = phone_number_entry.get()
       print("entered phone number =",phone_number_entered)
       print("user/id enter ",user_name_std_id)
       print("Data type of phone_number_entered:", type(phone_number_entered))
       if not (user_name_std_id and phone_number_entered):
         print("Please enter both username/student ID and phone number")
         # self.T7.deiconify()
         username_stdid.delete(0,tk.END)
         phone_number_entry.delete(0,tk.END)
         return
       error = []
       if not phone_number_entered.isdigit():
         error.append("Phone number should contain only digits")
       if len(phone_number_entered) != 10:
         error.append("Phone number should be 10 digits")
       if error:
         print(error)
         username_stdid.delete(0, tk.END)
         phone_number_entry.delete(0, tk.END)
         print("Please go back and correct the errors")
         # self.T7.deiconify()
         username_stdid.delete(0,tk.END)
         phone_number_entry.delete(0,tk.END)
         return
       else:
          mydb, cursor = self.establish_connection()
          query_admin = "SELECT phone_number FROM admin_table WHERE user_name = %s"
          cursor.execute(query_admin, (user_name_std_id,))
          result_admin = cursor.fetchone()
          query_student = "SELECT phone_number FROM student_database WHERE std_id = %s"
          cursor.execute(query_student, (user_name_std_id,))
          result_student = cursor.fetchone()
          if result_admin:
           phone_number = result_admin[0]
           print("Phone Number (from admin_table):", phone_number)
           print("Data type of phone_number:", type(phone_number))
           if str(phone_number) == phone_number_entered:
               if not phone_number_entered.startswith("+91"):
                   phone_number_entered = "+91" + phone_number_entered
               otp = self.generate_otp()
               self.send_otp(phone_number_entered, otp)
               if otp:
                  self.T7.destroy()  
                  self.enter_opt(otp,user_name_std_id)         
           else:
              info="Phone u entered for admin was in correct"
              simpledialog.messagebox.showinfo("STUDENT Status", info)
              username_stdid.delete(0,tk.END)
              phone_number_entry.delete(0,tk.END)
              self.T7.deiconify()    
          elif result_student:
           phone_number = result_student[0]
           print("Phone Number (from student_database):", phone_number)
           print("Data type of phone_number:", type(phone_number))
           if str(phone_number) == phone_number_entered:
                  if not phone_number_entered.startswith("+91"):
                   phone_number_entered = "+91" + phone_number_entered
                  otp = self.generate_otp()
                  print("Data type of otp:", type(otp))
                  self.send_otp(phone_number_entered, otp)
                  if otp:
                   self.T7.destroy()
                   self.enter_opt(otp, user_name_std_id)         
           else:
              info="Phone u entered for student was in correct"
              simpledialog.messagebox.showinfo("STUDENT Status", info)
              username_stdid.delete(0,tk.END)
              phone_number_entry.delete(0,tk.END)
              self.T7.deiconify()   
          else:
            info="No phone number found for the provided user_id / Std _id"
            simpledialog.messagebox.showinfo("STUDENT Status", info)
            mydb.close()
            username_stdid.delete(0,tk.END)
            phone_number_entry.delete(0,tk.END)
            self.T7.deiconify()        
    def enter_opt(self,otp,user_name_std_id):
       self.T8 = tk.Tk()
       self.T8.title("forget")
       frame_opt=tk.Frame(self.T8,borderwidth=6, relief="raised")
       frame_opt.pack() 
       entry=tk.Entry(frame_opt,width=40)
       entry.grid(row=0,columnspan=4,padx=10,pady=10)
       submit = tk.Button(frame_opt, text="submit opt", borderwidth=6, relief="raised", command=lambda:self.submit_opt(frame_opt,otp,entry,user_name_std_id))
       submit.grid(row=2, columnspan=4, padx=10, pady=10)
       Back=tk.Label(frame_opt,text='forgot_user_name',font=("Times New Roman", 4))
       Back.grid(row=0,column=0,padx=10,pady=10)
       Back.bind("<Enter>", lambda event: Back.config(font=("Times New Roman", 12, "underline")))
       Back.bind("<Leave>", lambda event: Back.config(font=("Times New Roman", 11)))
       Back.bind("<Button-1>", lambda event: self.back(self.T8))
       self.T8.protocol("WM_DELETE_WINDOW", self.on_closing)
       self.T8.mainloop()
       print(otp)
    def generate_otp(self):
        return str(random.randint(1000, 9999))
    def send_otp(self, to_number, otp):
        account_sid = 'ACc2784ae2f1756e3dab652f03e3826776'
        auth_token = 'c9e3c5a3b8206b25148fa6f9baa0883d'
        twilio_number = '+16202548140'
        client = Client(account_sid, auth_token)
        message = client.messages.create(
            body=f"Your OTP is: {otp}",
            from_=twilio_number,
            to=to_number
        )
        print("OTP sent successfully!")  
    def submit_opt(self,frame_opt,otp,entry,user_name_std_id):
       entered_opt=entry.get()
       print("Entered_opt",entered_opt)
       if entered_opt==otp:
          self.clear_frame(frame_opt)
          enter_password=tk.Label(frame_opt,text="Enter New password",font=("Times New Roman", 12))
          enter_password.grid(row=0,column=0,padx=10,pady=10)
          enter_new_password=tk.Entry(frame_opt,width=30)
          enter_new_password.grid(row=0,column=1,padx=10,pady=10)
          new_set_password_button=tk.Button(frame_opt,text="submit",borderwidth=6, relief="raised", command=lambda:self.new_set_password(enter_new_password,user_name_std_id))
          new_set_password_button.grid(row=2,columnspan=4,padx=10,pady=10)
       else:
          info="entered opt is incorrect"
          simpledialog.messagebox.showinfo("STUDENT Status", info)
          entry.delete(0,tk.END)
    def new_set_password(self,enter_new_password,user_name_std_id):
       new_password=enter_new_password.get()
       if not enter_new_password.get():
          info="password can't be empty"
          simpledialog.messagebox.showinfo("STUDENT Status", info)
          return
       try:
          mydb, cursor = self.establish_connection()
          query = "UPDATE login_credentials SET password = %s WHERE login_id = %s"
          cursor.execute(query, (new_password,user_name_std_id))
          mydb.commit()
          if cursor.rowcount >0:
             info= f"successfully changed password for the User_name / std_id = {user_name_std_id}"
             simpledialog.messagebox.showinfo("STUDENT Status", info)
             if hasattr(self, 'T8') and self.T8.winfo_exists():
                self.T8.destroy()
                print("T8 destroyed ")
             if hasattr(self, 'T2') and self.T2.winfo_exists():
                     self.T2.deiconify()
             if hasattr(self, 'T4') and self.T4.winfo_exists():
                  self.T4.deiconify()
             print("admin login page ")
          else:
             info="unable to change the pass word " 
             simpledialog.messagebox.showinfo("STUDENT Status", info)
       except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("SQL ERROR", error)
            enter_new_password.delete(0,tk.END)
       finally:
            if 'mydb' in locals():
              mydb.close()
            if 'cursor' in locals():  
              cursor.close()        
    def forget_user_name(self,event=None):
       if hasattr(self, 'T2') and self.T2.winfo_exists():
         self.T2.withdraw()
       if hasattr(self, 'T4') and self.T4.winfo_exists():
         self.T4.withdraw()
       self.T9=tk.Tk()
       self.T9.title("forget User name")
       username_frame=tk.Frame(self.T9,borderwidth=6, relief="raised")
       username_frame.pack()
       Back=tk.Label(username_frame,text='forgot_user_name',font=("Times New Roman", 10))
       Back.grid(row=0,column=0,padx=10,pady=10)
       Back.bind("<Enter>", lambda event: Back.config(font=("Times New Roman", 12, "underline")))
       Back.bind("<Leave>", lambda event: Back.config(font=("Times New Roman", 11)))
       Back.bind("<Button-1>", lambda event: self.back(self.T9))
       username_label=tk.Label(username_frame,text=" Admin Name / Student Name",font=("Times New Roman", 12))
       username_label.grid(row=0,column=1,padx=10,pady=10)
       username_entery=tk.Entry(username_frame,width=30)
       username_entery.grid(row=0,column=2,padx=10,pady=10)
       user_phonenumber=tk.Label(username_frame,text=" Admin Phonenumber / Student Phonenumber ",font=("Times New Roman", 12))
       user_phonenumber.grid(row=2,column=0,padx=10,pady=10)
       user_phonenumber_entery=tk.Entry(username_frame,width=30)
       user_phonenumber_entery.grid(row=2,column=1,padx=10,pady=10)
       get_username=tk.Button(username_frame,text=" Get User Name ",borderwidth=6, relief="raised",command=lambda:self.get_username(username_entery,user_phonenumber_entery))
       get_username.grid(row=4,columnspan=4 ,padx=10,pady=10)
       self.T9.protocol("WM_DELETE_WINDOW", self.on_closing)
       self.T9.mainloop()
    def get_username(self,username_entery,user_phonenumber_entery):  
       if not(username_entery.get() and user_phonenumber_entery.get()):
          info="All fields should be filled"
          simpledialog.messagebox.showinfo("STUDENT Status", info)
       error_user=[]
       if not re.match("^[a-zA-Z\s\W]+$", username_entery.get()):
          error_user.append("user Name can't have the Number ")
       if not user_phonenumber_entery.get().isdigit():
          error_user.append("Phone Number should only contain Number")
       if  len(user_phonenumber_entery.get()) !=10:
          error_user.append("Phone should be of 10 digits!")
       if error_user:
          tk.messagebox.showerror("Error", "\n".join(error_user))
          username_entery.delete(0,tk.END)
          user_phonenumber_entery.delete(0,tk.END)
          username_entery.focus_set()
          return
       else:
         name=username_entery.get()
         phone_number=user_phonenumber_entery.get()
         try:
            mydb, cursor = self.establish_connection()
            query_admin = "SELECT user_name FROM admin_table WHERE name = %s and phone_number = %s"
            cursor.execute(query_admin, (name,phone_number))
            result_admin = cursor.fetchone()
            query_student = "SELECT std_id FROM student_database WHERE std_name = %s and phone_number= %s"
            cursor.execute(query_student, (name,phone_number))
            result_student = cursor.fetchone()
            if result_admin:
                User_name = result_admin[0]
                simpledialog.messagebox.showinfo("User Name", User_name)
            elif result_student:
               std_id = result_student[0]
               simpledialog.messagebox.showinfo("Student ID", std_id)
            else:
               simpledialog.messagebox.showinfo("Information", "No matching record found.")
            if result_admin:
               if not phone_number.startswith("+91"):
                 phone_number = "+91" + phone_number
               self.send_otp(phone_number, User_name)
            elif result_student:
               if not phone_number.startswith("+91"):
                  phone_number = "+91" + phone_number
               self.send_otp(phone_number, std_id)
            if hasattr(self, 'T2') and self.T2.winfo_exists():
               self.T2.deiconify()
               self.T9.destroy()
            if hasattr(self, 'T4') and self.T4.winfo_exists():
               self.T4.deiconify()
               self.T9.destroy()
         except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("SQL ERROR", error)
            username_entery.delete(0,tk.END)
            user_phonenumber_entery.delete(0,tk.END)
            username_entery.focus_set()
         except twilio.base.exceptions.TwilioRestException as twilio_error:
            error_message = f"Twilio Error: {twilio_error}"
            simpledialog.messagebox.showinfo("Twilio Error", error_message)
            if hasattr(self, 'T2') and self.T2.winfo_exists():
               self.T2.deiconify()
               self.T9.destroy()
            if hasattr(self, 'T4') and self.T4.winfo_exists():
               self.T4.deiconify()
               self.T9.destroy()
         finally:
            if 'mydb' in locals():
              mydb.close()
            if 'cursor' in locals():  
              cursor.close()       
    def change_password(self):
       if hasattr(self, 'T3') and self.T3.winfo_exists():
                  self.T3.destroy()
       if hasattr(self, 'T5') and self.T5.winfo_exists():
                  self.T5.destroy()
       self.T6=tk.Tk()
       self.T6.title("Change_password")
       changepassword=ttk.Frame(self.T6,borderwidth=6, relief="raised")
       changepassword.pack()
       user_name=tk.Label(changepassword,text="User Name",font=("Times New Roman", 12))
       user_name.grid(row=0,column=0,padx=10,pady=10)
       user_name_entry=tk.Entry(changepassword,width=30)
       user_name_entry.grid(row=0,column=1,padx=10,pady=10)
       old_password_label=tk.Label(changepassword,text="old_Password",font=("Times New Roman", 12))
       old_password_label.grid(row=2,column=0,padx=10,pady=10)
       old_password=tk.Entry(changepassword,width=30,show="*")
       old_password.grid(row=2,column=1,padx=10,pady=10)
       new_password_label=tk.Label(changepassword,text="new_password",font=("Times New Roman", 12))
       new_password_label.grid(row=3,column=0,padx=10,pady=10)
       new_password=tk.Entry(changepassword,width=30,show="*")
       new_password.grid(row=3,column=1,padx=10,pady=10)
       login_button=tk.Button(changepassword,text="changepassword",font=("Times New Roman", 12),command=lambda: self.making_passwordchange(user_name_entry,old_password,new_password,T6))
       login_button.grid(row=5,columnspan=4,padx=10,pady=10)
       self.T9.protocol("WM_DELETE_WINDOW", self.p_u_on_closing)
       self.T6.mainloop()  
    def making_passwordchange(self,user_name_entry,old_password,new_password,T6):
        if not(user_name_entry.get()and old_password.get(),new_password.get()):
           messages="Any Filed  Can't be Empty"
           simpledialog.messagebox.showinfo("error", messages)
           return
        user_name_value = user_name_entry.get()
        old_password_value = old_password.get()
        new_password_value = new_password.get()
        try:
           mydb, cursor = self.establish_connection()
           query= "UPDATE login_credentials SET password = %s WHERE login_id = %s AND password = %s"
           cursor.execute(query,(new_password_value,user_name_value,old_password_value))
           mydb.commit()
           user_name_entry.delete(0, tk.END)
           old_password.delete(0, tk.END)
           new_password.delete(0, tk.END)
           if cursor.rowcount > 0:
            simpledialog.messagebox.showinfo("Password Change", "Password changed successfully")
            T6.destroy()
            if hasattr(self, 'T2') and self.T2.winfo_exists():
                  self.T2.deiconify()
            if hasattr(self, 'T4') and self.T4.winfo_exists():
                  self.T4.deiconify()
           else:
            simpledialog.messagebox.showinfo("Password Change", "Incorrect username or old password")
            user_name_entry.delete(0, tk.END)
            old_password.delete(0, tk.END)
            new_password.delete(0, tk.END)
            user_name_entry.focus_set()
        except mysql.connector.Error as error:
            simpledialog.messagebox.showinfo("Password Change", error)
            T6.destroy()
        finally:
           if 'mydb' in locals():
              mydb.close()
           if 'cursor' in locals():  
              cursor.close()  
    def clear_frame(self,frame):
      for widget in frame.winfo_children():
         widget.destroy()
    def establish_connection(self):
       try:
          mydb = mysql.connector.connect(
             host="localhost",
             user="root",
             password="Hari@25082002",
             database="library_database")
          cursor = mydb.cursor()
          return mydb, cursor
       except mysql.connector.Error as error:
          print("Error while connecting to MySQL", error)
          raise
    def logout(self, window):
       confirm_logout = messagebox.askquestion("Confirm Logout", "Are you sure you want to logout?")
       if confirm_logout == 'yes':
             window.destroy() 
             if hasattr(self, 'T3') and window == self.T3:
               self.T2.deiconify()
             elif hasattr(self, 'T5') and window == self.T5:
                self.T4.deiconify()
       else:
          pass
    def back_to_home(self,window):
       confirm_logout = messagebox.askquestion("Confirm Logout", "Are you sure you want to logout?")
       if confirm_logout == 'yes':
          window.destroy()
          new_smart_library_instance = SmartLibrary()
       else:
          pass
    def disable_widgets(self,frame):
       for widget in frame.winfo_children():
          widget.config(state="disabled")
    def on_closing(self):
       pass
    def back(self,window,event=None):
      if hasattr(self, 'T2') and self.T2.winfo_exists():
         self.T2.destroy()
      if hasattr(self, 'T4') and self.T4.winfo_exists():
         self.T4.destroy()
      if window.winfo_exists():
         window.destroy()
      self.__init__()
      
library = SmartLibrary()


